In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wlasl-processed/nslt_2000.json
/kaggle/input/wlasl-processed/nslt_1000.json
/kaggle/input/wlasl-processed/WLASL_v0.3.json
/kaggle/input/wlasl-processed/wlasl_class_list.txt
/kaggle/input/wlasl-processed/nslt_300.json
/kaggle/input/wlasl-processed/missing.txt
/kaggle/input/wlasl-processed/nslt_100.json
/kaggle/input/wlasl-processed/videos/03238.mp4
/kaggle/input/wlasl-processed/videos/19110.mp4
/kaggle/input/wlasl-processed/videos/24027.mp4
/kaggle/input/wlasl-processed/videos/65091.mp4
/kaggle/input/wlasl-processed/videos/08595.mp4
/kaggle/input/wlasl-processed/videos/56178.mp4
/kaggle/input/wlasl-processed/videos/57934.mp4
/kaggle/input/wlasl-processed/videos/32478.mp4
/kaggle/input/wlasl-processed/videos/54425.mp4
/kaggle/input/wlasl-processed/videos/42670.mp4
/kaggle/input/wlasl-processed/videos/59051.mp4
/kaggle/input/wlasl-processed/videos/65405.mp4
/kaggle/input/wlasl-processed/videos/30278.mp4
/kaggle/input/wlasl-processed/videos/03318.mp4
/kaggle/input/wlasl-proce

In [2]:
!pip install mediapipe==0.10.21 --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 51.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 4.25.8 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is in

In [3]:
# ============================================================
# 🧹 CLEAN SETUP
# ============================================================
import os, warnings, logging
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GLOG_minloglevel"] = "3"
os.environ["MEDIAPIPE_DISABLE_GPU_FEEDBACK"] = "1"
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

# ============================================================
# 🧠 IMPORTS
# ============================================================
import cv2
import numpy as np
import mediapipe as mp
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import json
import glob
import pandas as pd

# ============================================================
# 📁 LOAD DATASET METADATA
# ============================================================
data_dir = "/kaggle/input/wlasl-processed"
json_path = os.path.join(data_dir, "WLASL_v0.3.json")

with open(json_path, "r") as f:
    data = json.load(f)

# Collect video-label mappings
videos, labels = [], []
for entry in data:
    gloss = entry["gloss"]
    for inst in entry["instances"]:
        vid_path = os.path.join(data_dir, "videos", inst["video_id"] + ".mp4")
        if os.path.exists(vid_path):
            videos.append(vid_path)
            labels.append(gloss)

print(f"✅ Found {len(videos)} valid videos for training.")

# Limit for demo if needed
videos, labels = videos[:1000], labels[:1000]

# ============================================================
# 🧩 DATASET DEFINITION
# ============================================================
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh

class SignDataset(Dataset):
    def __init__(self, video_paths, labels):
        self.videos = video_paths
        self.labels = labels
        self.label_to_idx = {lbl: i for i, lbl in enumerate(sorted(set(labels)))}
        self.pose = mp_pose.Pose(static_image_mode=False)
        self.hands = mp_hands.Hands(static_image_mode=False)
        self.face = mp_face_mesh.FaceMesh(static_image_mode=False)

    def extract_features(self, frame):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res_pose = self.pose.process(frame_rgb)
        res_hands = self.hands.process(frame_rgb)
        res_face = self.face.process(frame_rgb)

        features = []
        if res_pose.pose_landmarks:
            features.extend([[lm.x, lm.y, lm.z] for lm in res_pose.pose_landmarks.landmark])
        if res_hands.left_hand_landmarks:
            features.extend([[lm.x, lm.y, lm.z] for lm in res_hands.left_hand_landmarks.landmark])
        if res_hands.right_hand_landmarks:
            features.extend([[lm.x, lm.y, lm.z] for lm in res_hands.right_hand_landmarks.landmark])
        if res_face.multi_face_landmarks:
            for lm in res_face.multi_face_landmarks[0].landmark:
                features.append([lm.x, lm.y, lm.z])
        return np.array(features).flatten()

    def __getitem__(self, idx):
        path = self.videos[idx]
        cap = cv2.VideoCapture(path)
        frames = []
        for _ in range(5):  # sample a few frames
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(self.extract_features(frame))
        cap.release()
        X = np.mean(frames, axis=0)
        y = self.label_to_idx[self.labels[idx]]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y)

    def __len__(self):
        return len(self.videos)

# ============================================================
# ⚙️ DATALOADERS
# ============================================================
dataset = SignDataset(videos, labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=2)

# ============================================================
# 🧠 MODEL DEFINITION
# ============================================================
class SignSenseModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SignSenseModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=2, dropout=0.3)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(1)
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

# ============================================================
# ⚡ TRAINING SETUP
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(labels))
input_size = 33*3 + 21*3 + 21*3 + 468*3  # pose + hands + face

model = SignSenseModel(input_size, 512, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scaler = torch.cuda.amp.GradScaler()
epochs = 5

print(f"\n🚀 Training on {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}) with {num_classes} classes")
print("=" * 70)

# ============================================================
# 🔁 TRAINING LOOP
# ============================================================
train_losses, test_accs = [], []
best_acc = 0.0

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    print(f"\n🟢 Epoch {epoch + 1}/{epochs} started...\n")

    for X_batch, y_batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}", leave=False):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast("cuda"):
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    # --- Evaluation ---
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_test, y_test in tqdm(test_loader, desc=f"Evaluating Epoch {epoch+1}", leave=False):
            X_test, y_test = X_test.to(device), y_test.to(device)
            with torch.amp.autocast("cuda"):
                preds = model(X_test)
            _, predicted = torch.max(preds, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()

    acc = 100 * correct / total
    test_accs.append(acc)

    print(f"✅ Epoch [{epoch+1}/{epochs}] complete.")
    print(f"   → Average Loss   : {avg_loss:.4f}")
    print(f"   → Test Accuracy  : {acc:.2f}%")
    print("-" * 70)

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "/kaggle/working/best_signsense_model.pth")
        print(f"💾 Best model saved (Accuracy: {best_acc:.2f}%)\n")

print("\n🎯 Training complete!")
print(f"🏆 Best accuracy achieved: {best_acc:.2f}%")
print("=" * 70)


E0000 00:00:1762607913.184980      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762607913.235025      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Found 11980 valid videos for training.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1762607940.471799     131 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762607940.494088     132 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762607940.503219     130 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762607940.540483     129 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762607940.568095     123 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762607940.627416     


🚀 Training on cuda (Tesla T4) with 99 classes

🟢 Epoch 1/5 started...



Training Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=0)


In [5]:
print("Torch using CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
else:
    print("⚠️ CUDA not active — training will run on CPU instead.")


Torch using CUDA: True
Device: Tesla T4


In [6]:
device = torch.device("cpu")


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Now running on:", device)
if device.type == "cuda":
    print("GPU Name:", torch.cuda.get_device_name(0))



✅ Now running on: cuda
GPU Name: Tesla T4


In [8]:
# ============================================================
# ✅ GPU-Safe + Log-Clean Training Loop for Kaggle T4 GPU
# ============================================================

import os, warnings, logging
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GLOG_minloglevel"] = "3"
os.environ["MEDIAPIPE_DISABLE_GPU_FEEDBACK"] = "1"
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ==========================================
# ⚙️ Device Setup
# ==========================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Torch using CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ Running on CPU fallback mode")

# ==========================================
# ⚙️ Model Definition
# ==========================================
class SignSenseModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SignSenseModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=2, dropout=0.3)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

# ==========================================
# ⚙️ Model, Optimizer, and Loss
# ==========================================
input_size = 33*3 + 21*3 + 21*3 + 468*3
num_classes = len(set(labels))
model = SignSenseModel(input_size, 512, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 5

# ============================================================
# 🧩 DataLoader (num_workers=0 for MediaPipe stability)
# ============================================================
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False, num_workers=0)

# ============================================================
# 🚀 Training Loop with Progress and Logs
# ============================================================
print(f"\n🚀 Training on {device} ({torch.cuda.get_device_name(0)}) with {num_classes} classes")
print("=" * 70)

train_losses, test_accs = [], []
best_acc = 0.0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
    for X_batch, y_batch in pbar:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Evaluate
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            preds = model(X_test)
            _, predicted = torch.max(preds, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()

    acc = 100 * correct / total
    test_accs.append(acc)
    print(f"✅ Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f} | Accuracy: {acc:.2f}%")

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "/kaggle/working/best_signsense_model.pth")

print("\n🎯 Training complete!")
print(f"🏆 Best Accuracy Achieved: {best_acc:.2f}%")

# ============================================================
# 📊 Plot Results
# ============================================================
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(test_accs, label='Accuracy')
plt.legend()
plt.show()

# Save log
pd.DataFrame({
    "epoch": range(1, epochs+1),
    "loss": train_losses,
    "accuracy": test_accs
}).to_csv("/kaggle/working/training_log.csv", index=False)

print("\n🧾 Log saved at /kaggle/working/training_log.csv")


Torch using CUDA: True
✅ GPU: Tesla T4

🚀 Training on cuda (Tesla T4) with 99 classes


Epoch 1/5:   0%|          | 0/100 [00:00<?, ?it/s]

W0000 00:00:1762608199.742480     125 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


AttributeError: type object 'SolutionOutputs' has no attribute 'left_hand_landmarks'

In [9]:
!pip install --quiet mediapipe opencv-python torch torchvision torchaudio tqdm numpy pandas scikit-learn matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 28.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's de

In [10]:
import os
import cv2
import numpy as np
import mediapipe as mp
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Suppress Mediapipe & TensorFlow logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GLOG_minloglevel"] = "3"
os.environ["MEDIAPIPE_DISABLE_GPU_FEEDBACK"] = "1"

# ✅ Confirm GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Torch using CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ Running on CPU fallback mode")


Torch using CUDA: True
✅ GPU: Tesla T4


In [11]:
class SignDataset(Dataset):
    def __init__(self, videos, labels):
        self.videos = videos
        self.labels = labels
        self.pose = mp.solutions.pose.Pose(static_image_mode=False)
        self.hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2)
        self.face = mp.solutions.face_mesh.FaceMesh(static_image_mode=False)
    
    def __len__(self):
        return len(self.videos)
    
    def extract_features(self, frame):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res_pose = self.pose.process(frame_rgb)
        res_hands = self.hands.process(frame_rgb)
        res_face = self.face.process(frame_rgb)

        features = []

        # ✅ Pose (33)
        if res_pose.pose_landmarks:
            features.extend([[lm.x, lm.y, lm.z] for lm in res_pose.pose_landmarks.landmark])
        else:
            features.extend([[0, 0, 0]] * 33)

        # ✅ Hands (21 × 2)
        hand_points = []
        if res_hands.multi_hand_landmarks:
            for hand_landmarks in res_hands.multi_hand_landmarks:
                hand_points.extend([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
            # Pad missing hand
            if len(res_hands.multi_hand_landmarks) == 1:
                hand_points.extend([[0, 0, 0]] * 21)
        else:
            hand_points.extend([[0, 0, 0]] * 42)
        features.extend(hand_points)

        # ✅ Face (468)
        if res_face.multi_face_landmarks:
            features.extend([[lm.x, lm.y, lm.z] for lm in res_face.multi_face_landmarks[0].landmark])
        else:
            features.extend([[0, 0, 0]] * 468)

        return np.array(features).flatten()

    def __getitem__(self, idx):
        video_path = self.videos[idx]
        label = self.labels[idx]
        cap = cv2.VideoCapture(video_path)
        frames = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(self.extract_features(frame))

        cap.release()

        if len(frames) == 0:
            X = np.zeros((1, 33*3 + 21*3 + 21*3 + 468*3))
        else:
            X = np.mean(frames, axis=0)
        return torch.tensor(X, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


In [12]:
import glob
from sklearn.preprocessing import LabelEncoder

# Path to dataset
data_dir = "/kaggle/input/wlasl-processed"

# Get all videos (recursive)
videos = glob.glob(os.path.join(data_dir, "videos/*/*.mp4"))
labels = [os.path.basename(os.path.dirname(v)) for v in videos]

print(f"✅ Found {len(videos)} valid labeled videos")

# Encode labels numerically
le = LabelEncoder()
labels = le.fit_transform(labels)

# Optional: limit for faster training in Kaggle (change 1000 → higher if needed)
videos, labels = videos[:1000], labels[:1000]

# Create dataset object
dataset = SignDataset(videos, labels)

# Split into 80% train, 20% test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False, num_workers=0)

print(f"✅ Train size: {len(train_ds)}, Test size: {len(test_ds)}, Classes: {len(set(labels))}")


✅ Found 0 valid labeled videos


W0000 00:00:1762608489.396228     192 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608489.421273     191 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608489.444229     189 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


ValueError: num_samples should be a positive integer value, but got num_samples=0

W0000 00:00:1762608489.479811     190 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608489.510831     183 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608489.562843     183 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [13]:
import os

data_dir = "/kaggle/input/wlasl-processed"

for root, dirs, files in os.walk(data_dir):
    print(f"{root} | dirs: {dirs[:5]} | files: {files[:5]}")
    break  # only show top level


/kaggle/input/wlasl-processed | dirs: ['videos'] | files: ['nslt_2000.json', 'nslt_1000.json', 'WLASL_v0.3.json', 'wlasl_class_list.txt', 'nslt_300.json']


In [14]:
import glob
from sklearn.preprocessing import LabelEncoder

# ✅ Dataset root directory
data_dir = "/kaggle/input/wlasl-processed"
video_dir = os.path.join(data_dir, "videos")

# ✅ Recursively search all .mp4 files under /videos/
videos = glob.glob(os.path.join(video_dir, "*", "*.mp4"), recursive=True)
labels = [os.path.basename(os.path.dirname(v)) for v in videos]

print(f"✅ Total videos found: {len(videos)}")
print(f"✅ Unique labels: {len(set(labels))}")

# 🧠 Optional: limit to 1000 samples for quicker demo training
videos, labels = videos[:1000], labels[:1000]

# Encode labels numerically
le = LabelEncoder()
labels = le.fit_transform(labels)

# ✅ Build dataset and loaders
dataset = SignDataset(videos, labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False, num_workers=0)

print(f"✅ Train size: {len(train_ds)}, Test size: {len(test_ds)}, Classes: {len(set(labels))}")


✅ Total videos found: 0
✅ Unique labels: 0


W0000 00:00:1762608575.375423     209 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


ValueError: num_samples should be a positive integer value, but got num_samples=0

W0000 00:00:1762608575.398902     207 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608575.427850     203 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608575.454809     203 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608575.460604     199 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608575.517757     202 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [15]:
import os

video_root = "/kaggle/input/wlasl-processed/videos"
for root, dirs, files in os.walk(video_root):
    print(f"{root} | dirs: {dirs[:5]} | files: {files[:5]}")
    break  # only show one level for now


/kaggle/input/wlasl-processed/videos | dirs: [] | files: ['03238.mp4', '19110.mp4', '24027.mp4', '65091.mp4', '08595.mp4']


In [16]:
import os, json, glob
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, random_split

data_dir = "/kaggle/input/wlasl-processed"
video_dir = os.path.join(data_dir, "videos")
json_path = os.path.join(data_dir, "WLASL_v0.3.json")

# ✅ Load JSON metadata
with open(json_path, "r") as f:
    data = json.load(f)

# Build mapping from video_id → gloss (label)
id_to_label = {}
for entry in data:
    gloss = entry["gloss"]
    for inst in entry["instances"]:
        vid = inst["video_id"]
        id_to_label[str(vid)] = gloss

# ✅ Match available videos to JSON labels
videos = glob.glob(os.path.join(video_dir, "*.mp4"))
labels = []

for v in videos:
    vid_id = os.path.splitext(os.path.basename(v))[0]
    label = id_to_label.get(vid_id)
    if label:
        labels.append(label)
    else:
        labels.append("unknown")  # fallback if not in JSON

# Filter out unknowns
paired = [(v, l) for v, l in zip(videos, labels) if l != "unknown"]
videos, labels = zip(*paired) if paired else ([], [])

print(f"✅ Total JSON entries: {len(data)}")
print(f"✅ Found {len(videos)} valid labeled videos")
print(f"✅ Unique gloss labels: {len(set(labels))}")
print("Example labels:", list(set(labels))[:10])

# Encode labels
le = LabelEncoder()
labels = le.fit_transform(labels)

# Optional: Limit to smaller subset for quick training
videos, labels = videos[:1000], labels[:1000]

# ✅ Build dataset & loaders
dataset = SignDataset(videos, labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_ds, test_ds = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False, num_workers=0)

print(f"\n✅ Train size: {len(train_ds)}, Test size: {len(test_ds)}, Classes: {len(set(labels))}")


✅ Total JSON entries: 2000
✅ Found 11980 valid labeled videos
✅ Unique gloss labels: 2000
Example labels: ['off', 'important', 'jealous', 'onion', 'drug', 'pack', 'popcorn', 'potato', 'joy', 'rainbow']

✅ Train size: 800, Test size: 200, Classes: 806


W0000 00:00:1762608641.108285     221 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608641.130288     221 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608641.154318     219 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608641.179702     216 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608641.184863     213 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608641.247031     213 inference_feedback_manager.cc:114] Feedback manager 

In [17]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# ===============================
#   SIGN SENSE LSTM MODEL
# ===============================
class SignSenseModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SignSenseModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=2, dropout=0.3)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

# ===============================
#   TRAINING SETUP
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Training on: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")

input_size = 33*3 + 21*3 + 21*3 + 468*3  # pose + hands + face features
hidden_size = 512
num_classes = len(set(labels))

model = SignSenseModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 5

train_losses, test_accs = [], []

print(f"Total Classes: {num_classes}")
print("="*70)

# ===============================
#   TRAINING LOOP
# ===============================
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    # ===========================
    #   EVALUATION
    # ===========================
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            preds = model(X_test)
            _, predicted = torch.max(preds, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()
    acc = 100 * correct / total
    test_accs.append(acc)

    print(f"✅ Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {acc:.2f}%")

# ===============================
#   VISUALIZATION
# ===============================
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(test_accs, label='Accuracy')
plt.legend()
plt.show()

# ===============================
#   SAVE TRAINING LOG
# ===============================
df = pd.DataFrame({"epoch": list(range(1, epochs+1)), "loss": train_losses, "accuracy": test_accs})
df.to_csv("/kaggle/working/training_log.csv", index=False)

print("\n✅ Training complete! Log saved at /kaggle/working/training_log.csv")


🚀 Training on: cuda (Tesla T4)
Total Classes: 806


Epoch 1/5:   0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: size mismatch (got input: [806], target: [8])

In [18]:
# =====================================
# ✅ SETUP
# =====================================
import os
import cv2
import json
import glob
import numpy as np
import mediapipe as mp
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GLOG_minloglevel"] = "3"
os.environ["MEDIAPIPE_DISABLE_GPU_FEEDBACK"] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Torch using CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ Running on CPU fallback mode")

# =====================================
# ✅ DATASET CLASS
# =====================================
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_mesh

class SignDataset(Dataset):
    def __init__(self, videos, labels, max_frames=20):
        self.videos = videos
        self.labels = labels
        self.max_frames = max_frames

    def extract_features(self, frame):
        features = []
        with mp_pose.Pose(static_image_mode=False) as pose, \
             mp_hands.Hands(static_image_mode=False, max_num_hands=2) as hands, \
             mp_face.FaceMesh(static_image_mode=False, max_num_faces=1) as face:

            res_pose = pose.process(frame)
            res_hands = hands.process(frame)
            res_face = face.process(frame)

            if res_pose.pose_landmarks:
                features.extend([[lm.x, lm.y, lm.z] for lm in res_pose.pose_landmarks.landmark])
            if res_hands.multi_hand_landmarks:
                for hand in res_hands.multi_hand_landmarks:
                    features.extend([[lm.x, lm.y, lm.z] for lm in hand.landmark])
            if res_face.multi_face_landmarks:
                for face_lm in res_face.multi_face_landmarks:
                    features.extend([[lm.x, lm.y, lm.z] for lm in face_lm.landmark])
        return np.array(features).flatten() if features else np.zeros((33*3 + 21*3 + 21*3 + 468*3))

    def __getitem__(self, idx):
        video_path = self.videos[idx]
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_count = 0
        while cap.isOpened() and frame_count < self.max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            features = self.extract_features(frame_rgb)
            frames.append(features)
            frame_count += 1
        cap.release()

        if not frames:
            frames = [np.zeros((33*3 + 21*3 + 21*3 + 468*3))]

        X = np.stack(frames)
        y = self.labels[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.videos)

# =====================================
# ✅ LOAD DATA
# =====================================
data_dir = "/kaggle/input/wlasl-processed"
video_dir = os.path.join(data_dir, "videos")
json_path = os.path.join(data_dir, "WLASL_v0.3.json")

with open(json_path, "r") as f:
    data = json.load(f)

id_to_label = {}
for entry in data:
    gloss = entry["gloss"]
    for inst in entry["instances"]:
        vid = inst["video_id"]
        id_to_label[str(vid)] = gloss

videos = glob.glob(os.path.join(video_dir, "*.mp4"))
labels = []
for v in videos:
    vid_id = os.path.splitext(os.path.basename(v))[0]
    label = id_to_label.get(vid_id)
    if label:
        labels.append(label)

paired = [(v, l) for v, l in zip(videos, labels)]
videos, labels = zip(*paired) if paired else ([], [])
print(f"✅ Found {len(videos)} labeled videos with {len(set(labels))} unique classes.")

le = LabelEncoder()
labels = le.fit_transform(labels)
videos, labels = videos[:1000], labels[:1000]

dataset = SignDataset(videos, labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False, num_workers=0)

print(f"✅ Train size: {len(train_ds)}, Test size: {len(test_ds)}, Classes: {len(set(labels))}")

# =====================================
# ✅ MODEL
# =====================================
class SignSenseModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SignSenseModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=2, dropout=0.3)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

input_size = 33*3 + 21*3 + 21*3 + 468*3
num_classes = len(set(labels))
model = SignSenseModel(input_size, 512, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 3  # you can increase later

# =====================================
# ✅ TRAINING LOOP
# =====================================
train_losses, test_accs = [], []
print(f"\n🚀 Training on {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}) with {num_classes} classes")

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    # EVALUATION
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            preds = model(X_test)
            _, predicted = torch.max(preds, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()
    acc = 100 * correct / total
    test_accs.append(acc)

    print(f"✅ Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {acc:.2f}%")

# =====================================
# ✅ RESULTS
# =====================================
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(test_accs, label='Accuracy')
plt.legend()
plt.show()

df = pd.DataFrame({"epoch": list(range(1, epochs+1)), "loss": train_losses, "accuracy": test_accs})
df.to_csv("/kaggle/working/training_log.csv", index=False)
print("\n✅ Training complete! Log saved at /kaggle/working/training_log.csv")


Torch using CUDA: True
✅ GPU: Tesla T4
✅ Found 11980 labeled videos with 2000 unique classes.
✅ Train size: 800, Test size: 200, Classes: 806

🚀 Training on cuda (Tesla T4) with 806 classes


Epoch 1/3:   0%|          | 0/100 [00:00<?, ?it/s]

W0000 00:00:1762608786.151404     273 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608786.166143     276 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608786.187501     269 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608786.213872     269 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608786.238367     265 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608786.297647     267 inference_feedback_manager.cc:114] Feedback manager 

ValueError: all input arrays must have the same shape

In [19]:
feedback tensors.
W0000 00:00:1762608794.091845     734 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.116932     730 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.143357     730 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.167988     727 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.214854     727 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.310580     746 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.314038     745 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.327043     747 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.335853     745 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.353303     739 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608794.393269     739 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
/tmp/ipykernel_48/1639136870.py in <cell line: 0>()
    159     model.train()
    160     total_loss = 0.0
--> 161     for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
    162         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
    163         optimizer.zero_grad()

/usr/local/lib/python3.11/dist-packages/tqdm/notebook.py in __iter__(self)
    248         try:
    249             it = super().__iter__()
--> 250             for obj in it:
    251                 # return super(tqdm...) will not catch exception
    252                 yield obj

/usr/local/lib/python3.11/dist-packages/tqdm/std.py in __iter__(self)
   1179 
   1180         try:
-> 1181             for obj in iterable:
   1182                 yield obj
   1183                 # Update and possibly print the progressbar.

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py in __next__(self)
    706                 # TODO(https://github.com/pytorch/pytorch/issues/76750)
    707                 self._reset()  # type: ignore[call-arg]
--> 708             data = self._next_data()
    709             self._num_yielded += 1
    710             if (

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py in _next_data(self)
    762     def _next_data(self):
    763         index = self._next_index()  # may raise StopIteration
--> 764         data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
    765         if self._pin_memory:
    766             data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py in fetch(self, possibly_batched_index)
     48         if self.auto_collation:
     49             if hasattr(self.dataset, "__getitems__") and self.dataset.__getitems__:
---> 50                 data = self.dataset.__getitems__(possibly_batched_index)
     51             else:
     52                 data = [self.dataset[idx] for idx in possibly_batched_index]

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py in __getitems__(self, indices)
    418             return self.dataset.__getitems__([self.indices[idx] for idx in indices])  # type: ignore[attr-defined]
    419         else:
--> 420             return [self.dataset[self.indices[idx]] for idx in indices]
    421 
    422     def __len__(self):

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py in <listcomp>(.0)
    418             return self.dataset.__getitems__([self.indices[idx] for idx in indices])  # type: ignore[attr-defined]
    419         else:
--> 420             return [self.dataset[self.indices[idx]] for idx in indices]
    421 
    422     def __len__(self):

/tmp/ipykernel_48/1639136870.py in __getitem__(self, idx)
     80             frames = [np.zeros((33*3 + 21*3 + 21*3 + 468*3))]
     81 
---> 82         X = np.stack(frames)
     83         y = self.labels[idx]
     84         return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

/usr/local/lib/python3.11/dist-packages/numpy/core/shape_base.py in stack(arrays, axis, out, dtype, casting)
    447     shapes = {arr.shape for arr in arrays}
    448     if len(shapes) != 1:
--> 449         raise ValueError('all input arrays must have the same shape')
    450 
    451     result_ndim = arrays[0].ndim + 1

ValueError: all input arrays must have the same shape

SyntaxError: unmatched ']' (4230951243.py, line 2)

In [20]:
# =======================================================
# ✅ FIXED SIGN DATASET CLASS
# =======================================================
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset

class SignDataset(Dataset):
    def __init__(self, videos, labels, max_frames=20):
        self.videos = videos
        self.labels = labels
        self.max_frames = max_frames
        self.feature_dim = 33*3 + 21*3 + 21*3 + 468*3  # pose + hands + face

    def extract_features(self, frame):
        features = []
        with mp_pose.Pose(static_image_mode=False) as pose, \
             mp_hands.Hands(static_image_mode=False, max_num_hands=2) as hands, \
             mp_face.FaceMesh(static_image_mode=False, max_num_faces=1) as face:

            res_pose = pose.process(frame)
            res_hands = hands.process(frame)
            res_face = face.process(frame)

            if res_pose.pose_landmarks:
                features.extend([[lm.x, lm.y, lm.z] for lm in res_pose.pose_landmarks.landmark])
            if res_hands.multi_hand_landmarks:
                for hand in res_hands.multi_hand_landmarks:
                    features.extend([[lm.x, lm.y, lm.z] for lm in hand.landmark])
            if res_face.multi_face_landmarks:
                for face_lm in res_face.multi_face_landmarks:
                    features.extend([[lm.x, lm.y, lm.z] for lm in face_lm.landmark])

        # ✅ Ensure consistent feature size
        flat = np.array(features).flatten()
        if flat.size < self.feature_dim:
            pad = np.zeros(self.feature_dim - flat.size)
            flat = np.concatenate([flat, pad])
        elif flat.size > self.feature_dim:
            flat = flat[:self.feature_dim]
        return flat

    def __getitem__(self, idx):
        video_path = self.videos[idx]
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_count = 0

        while cap.isOpened() and frame_count < self.max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            features = self.extract_features(frame_rgb)
            frames.append(features)
            frame_count += 1
        cap.release()

        # ✅ Pad/truncate to fixed max_frames
        if len(frames) < self.max_frames:
            pad_frames = [np.zeros(self.feature_dim) for _ in range(self.max_frames - len(frames))]
            frames.extend(pad_frames)
        elif len(frames) > self.max_frames:
            frames = frames[:self.max_frames]

        X = np.stack(frames)
        y = self.labels[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.videos)


In [21]:
# =======================================================
# ✅ MODEL DEFINITION
# =======================================================
import torch.nn as nn

class SignSenseModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SignSenseModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=2, dropout=0.3)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

input_size = 33*3 + 21*3 + 21*3 + 468*3
num_classes = len(set(labels))
model = SignSenseModel(input_size, 512, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 5  # increase later if GPU time allows

print(f"\n🚀 Training on: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")
print(f"Total Classes: {num_classes}")
print("="*70)



🚀 Training on: cuda (Tesla T4)
Total Classes: 806


In [22]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd

train_losses, test_accs = [], []

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            preds = model(X_test)
            _, predicted = torch.max(preds, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()

    acc = 100 * correct / total
    test_accs.append(acc)
    print(f"✅ Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {acc:.2f}%")

# =======================================================
# ✅ RESULTS
# =======================================================
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(test_accs, label='Accuracy')
plt.legend()
plt.show()

df = pd.DataFrame({
    "epoch": list(range(1, epochs+1)),
    "loss": train_losses,
    "accuracy": test_accs
})
df.to_csv("/kaggle/working/training_log.csv", index=False)
print("\n✅ Training complete! Log saved at /kaggle/working/training_log.csv")


Epoch 1/5:   0%|          | 0/100 [00:00<?, ?it/s]

W0000 00:00:1762608912.341184     766 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608912.345737     763 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608912.353228     766 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608912.370319     763 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608912.405344     758 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762608912.460186     758 inference_feedback_manager.cc:114] Feedback manager 

ValueError: all input arrays must have the same shape

In [23]:
class SignDataset(Dataset):
    def __init__(self, videos, labels, max_frames=20):
        self.videos = videos
        self.labels = labels
        self.max_frames = max_frames
        self.feature_dim = 33*3 + 21*3 + 21*3 + 468*3  # 1629 total

    def extract_features(self, frame):
        # Preallocate fixed-size vector
        flat = np.zeros(self.feature_dim, dtype=np.float32)
        features = []
        with mp_pose.Pose(static_image_mode=False) as pose, \
             mp_hands.Hands(static_image_mode=False, max_num_hands=2) as hands, \
             mp_face.FaceMesh(static_image_mode=False, max_num_faces=1) as face:

            res_pose = pose.process(frame)
            res_hands = hands.process(frame)
            res_face = face.process(frame)

            if res_pose.pose_landmarks:
                features.extend([[lm.x, lm.y, lm.z] for lm in res_pose.pose_landmarks.landmark])
            if res_hands.multi_hand_landmarks:
                for hand in res_hands.multi_hand_landmarks:
                    features.extend([[lm.x, lm.y, lm.z] for lm in hand.landmark])
            if res_face.multi_face_landmarks:
                for face_lm in res_face.multi_face_landmarks:
                    features.extend([[lm.x, lm.y, lm.z] for lm in face_lm.landmark])

        # Fill available features into the preallocated vector
        flat_part = np.array(features, dtype=np.float32).flatten()
        flat[:min(flat.size, flat_part.size)] = flat_part[:flat.size]
        return flat

    def __getitem__(self, idx):
        video_path = self.videos[idx]
        cap = cv2.VideoCapture(video_path)
        frames = []

        while cap.isOpened() and len(frames) < self.max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            features = self.extract_features(frame_rgb)
            frames.append(features)
        cap.release()

        # Pad / truncate to ensure same number of frames
        if len(frames) < self.max_frames:
            pad_frames = [np.zeros(self.feature_dim, dtype=np.float32)
                          for _ in range(self.max_frames - len(frames))]
            frames.extend(pad_frames)
        elif len(frames) > self.max_frames:
            frames = frames[:self.max_frames]

        # Stack into consistent tensor
        X = np.stack(frames)
        y = self.labels[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.videos)


In [24]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd

train_losses, test_accs = [], []

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            preds = model(X_test)
            _, predicted = torch.max(preds, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()

    acc = 100 * correct / total
    test_accs.append(acc)
    print(f"✅ Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {acc:.2f}%")

# =======================================================
# ✅ RESULTS
# =======================================================
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(test_accs, label='Accuracy')
plt.legend()
plt.show()

df = pd.DataFrame({
    "epoch": list(range(1, epochs+1)),
    "loss": train_losses,
    "accuracy": test_accs
})
df.to_csv("/kaggle/working/training_log.csv", index=False)
print("\n✅ Training complete! Log saved at /kaggle/working/training_log.csv")


Epoch 1/5:   0%|          | 0/100 [00:00<?, ?it/s]

W0000 00:00:1762609005.203025    1744 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609005.206616    1739 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609005.213114    1744 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609005.229017    1739 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609005.250096    1736 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609005.281918    1736 inference_feedback_manager.cc:114] Feedback manager 

ValueError: all input arrays must have the same shape

In [25]:
class SignDataset(Dataset):
    def __init__(self, videos, labels, max_frames=20):
        self.videos = videos
        self.labels = labels
        self.max_frames = max_frames
        self.feature_dim = 33*3 + 21*3 + 21*3 + 468*3  # 1629 features per frame

        # Initialize MediaPipe solutions once (faster)
        self.pose = mp.solutions.pose.Pose(static_image_mode=False)
        self.hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2)
        self.face = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

    def extract_features(self, frame):
        features = np.zeros((self.feature_dim,), dtype=np.float32)

        res_pose = self.pose.process(frame)
        res_hands = self.hands.process(frame)
        res_face = self.face.process(frame)

        temp = []

        # Pose landmarks
        if res_pose.pose_landmarks:
            temp.extend([[lm.x, lm.y, lm.z] for lm in res_pose.pose_landmarks.landmark])
        # Hands
        if res_hands.multi_hand_landmarks:
            for hand in res_hands.multi_hand_landmarks:
                temp.extend([[lm.x, lm.y, lm.z] for lm in hand.landmark])
        # Face
        if res_face.multi_face_landmarks:
            for face_lm in res_face.multi_face_landmarks:
                temp.extend([[lm.x, lm.y, lm.z] for lm in face_lm.landmark])

        temp = np.array(temp, dtype=np.float32).flatten()

        # Copy available features into the fixed-size array
        features[:min(self.feature_dim, len(temp))] = temp[:self.feature_dim]
        return features

    def __getitem__(self, idx):
        video_path = self.videos[idx]
        cap = cv2.VideoCapture(video_path)
        frames = []

        while cap.isOpened() and len(frames) < self.max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            features = self.extract_features(frame_rgb)
            frames.append(features)
        cap.release()

        # Ensure fixed number of frames
        while len(frames) < self.max_frames:
            frames.append(np.zeros((self.feature_dim,), dtype=np.float32))
        if len(frames) > self.max_frames:
            frames = frames[:self.max_frames]

        X = np.stack(frames)  # ✅ guaranteed consistent shapes now
        y = self.labels[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.videos)


In [26]:
# =======================================================
# ✅ MODEL DEFINITION
# =======================================================
import torch.nn as nn

class SignSenseModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SignSenseModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=2, dropout=0.3)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

input_size = 33*3 + 21*3 + 21*3 + 468*3
num_classes = len(set(labels))
model = SignSenseModel(input_size, 512, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 3  # increase later if GPU time allows

print(f"\n🚀 Training on: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")
print(f"Total Classes: {num_classes}")
print("="*70)



🚀 Training on: cuda (Tesla T4)
Total Classes: 806


In [27]:
class SignDataset(Dataset):
    def __init__(self, videos, labels, max_frames=20):
        self.videos = videos
        self.labels = labels
        self.max_frames = max_frames
        self.feature_dim = 33*3 + 21*3 + 21*3 + 468*3  # 1629 total

    def extract_features(self, frame):
        # Preallocate fixed-size vector
        flat = np.zeros(self.feature_dim, dtype=np.float32)
        features = []
        with mp_pose.Pose(static_image_mode=False) as pose, \
             mp_hands.Hands(static_image_mode=False, max_num_hands=2) as hands, \
             mp_face.FaceMesh(static_image_mode=False, max_num_faces=1) as face:

            res_pose = pose.process(frame)
            res_hands = hands.process(frame)
            res_face = face.process(frame)

            if res_pose.pose_landmarks:
                features.extend([[lm.x, lm.y, lm.z] for lm in res_pose.pose_landmarks.landmark])
            if res_hands.multi_hand_landmarks:
                for hand in res_hands.multi_hand_landmarks:
                    features.extend([[lm.x, lm.y, lm.z] for lm in hand.landmark])
            if res_face.multi_face_landmarks:
                for face_lm in res_face.multi_face_landmarks:
                    features.extend([[lm.x, lm.y, lm.z] for lm in face_lm.landmark])

        # Fill available features into the preallocated vector
        flat_part = np.array(features, dtype=np.float32).flatten()
        flat[:min(flat.size, flat_part.size)] = flat_part[:flat.size]
        return flat

    def __getitem__(self, idx):
        video_path = self.videos[idx]
        cap = cv2.VideoCapture(video_path)
        frames = []

        while cap.isOpened() and len(frames) < self.max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            features = self.extract_features(frame_rgb)
            frames.append(features)
        cap.release()

        # Pad / truncate to ensure same number of frames
        if len(frames) < self.max_frames:
            pad_frames = [np.zeros(self.feature_dim, dtype=np.float32)
                          for _ in range(self.max_frames - len(frames))]
            frames.extend(pad_frames)
        elif len(frames) > self.max_frames:
            frames = frames[:self.max_frames]

        # Stack into consistent tensor
        X = np.stack(frames)
        y = self.labels[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.videos)


In [28]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd

train_losses, test_accs = [], []

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            preds = model(X_test)
            _, predicted = torch.max(preds, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()

    acc = 100 * correct / total
    test_accs.append(acc)
    print(f"✅ Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {acc:.2f}%")

# =======================================================
# ✅ RESULTS
# =======================================================
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(test_accs, label='Accuracy')
plt.legend()
plt.show()

df = pd.DataFrame({
    "epoch": list(range(1, epochs+1)),
    "loss": train_losses,
    "accuracy": test_accs
})
df.to_csv("/kaggle/working/training_log.csv", index=False)
print("\n✅ Training complete! Log saved at /kaggle/working/training_log.csv")


Epoch 1/3:   0%|          | 0/100 [00:00<?, ?it/s]

W0000 00:00:1762609082.614497    1991 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609082.615152    1987 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609082.621065    1992 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609082.646380    1987 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609082.665262    1985 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609082.693301    1986 inference_feedback_manager.cc:114] Feedback manager 

ValueError: all input arrays must have the same shape

In [30]:
ValueError                                Traceback (most recent call last)
/tmp/ipykernel_48/1766287256.py in <cell line: 0>()
      9     total_loss = 0.0
     10 
---> 11     for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
     12         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
     13         optimizer.zero_grad()

/usr/local/lib/python3.11/dist-packages/tqdm/notebook.py in __iter__(self)
    248         try:
    249             it = super().__iter__()
--> 250             for obj in it:
    251                 # return super(tqdm...) will not catch exception
    252                 yield obj

/usr/local/lib/python3.11/dist-packages/tqdm/std.py in __iter__(self)
   1179 
   1180         try:
-> 1181             for obj in iterable:
   1182                 yield obj
   1183                 # Update and possibly print the progressbar.

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py in __next__(self)
    706                 # TODO(https://github.com/pytorch/pytorch/issues/76750)
    707                 self._reset()  # type: ignore[call-arg]
--> 708             data = self._next_data()
    709             self._num_yielded += 1
    710             if (

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py in _next_data(self)
    762     def _next_data(self):
    763         index = self._next_index()  # may raise StopIteration
--> 764         data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
    765         if self._pin_memory:
    766             data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py in fetch(self, possibly_batched_index)
     48         if self.auto_collation:
     49             if hasattr(self.dataset, "__getitems__") and self.dataset.__getitems__:
---> 50                 data = self.dataset.__getitems__(possibly_batched_index)
     51             else:
     52                 data = [self.dataset[idx] for idx in possibly_batched_index]

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py in __getitems__(self, indices)
    418             return self.dataset.__getitems__([self.indices[idx] for idx in indices])  # type: ignore[attr-defined]
    419         else:
--> 420             return [self.dataset[self.indices[idx]] for idx in indices]
    421 
    422     def __len__(self):

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py in <listcomp>(.0)
    418             return self.dataset.__getitems__([self.indices[idx] for idx in indices])  # type: ignore[attr-defined]
    419         else:
--> 420             return [self.dataset[self.indices[idx]] for idx in indices]
    421 
    422     def __len__(self):

/tmp/ipykernel_48/1639136870.py in __getitem__(self, idx)
     80             frames = [np.zeros((33*3 + 21*3 + 21*3 + 468*3))]
     81 
---> 82         X = np.stack(frames)
     83         y = self.labels[idx]
     84         return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

/usr/local/lib/python3.11/dist-packages/numpy/core/shape_base.py in stack(arrays, axis, out, dtype, casting)
    447     shapes = {arr.shape for arr in arrays}
    448     if len(shapes) != 1:
--> 449         raise ValueError('all input arrays must have the same shape')
    450 
    451     result_ndim = arrays[0].ndim + 1

ValueError: all input arrays must have the same shape

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

In [31]:
import json, cv2, torch
import numpy as np
from torch.utils.data import Dataset

with open("/kaggle/input/wlasl-processed/WLASL_v0.3.json", "r") as f:
    meta = json.load(f)

# Build mapping of video_id → metadata
video_meta = {}
for entry in meta:
    gloss = entry["gloss"]
    for inst in entry["instances"]:
        vid = inst["video_id"]
        video_meta[vid] = {
            "gloss": gloss,
            "bbox": inst.get("bbox", [0, 0, 0, 0]),
            "frame_start": inst.get("frame_start", 1),
            "frame_end": inst.get("frame_end", -1)
        }

class SignDataset(Dataset):
    def __init__(self, videos, labels, max_frames=20):
        self.videos = videos
        self.labels = labels
        self.max_frames = max_frames
        self.feature_dim = 33*3 + 21*3 + 21*3 + 468*3

        self.pose = mp.solutions.pose.Pose(static_image_mode=False)
        self.hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2)
        self.face = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

    def extract_features(self, frame):
        flat = np.zeros((self.feature_dim,), dtype=np.float32)
        res_pose = self.pose.process(frame)
        res_hands = self.hands.process(frame)
        res_face = self.face.process(frame)

        feats = []
        if res_pose.pose_landmarks:
            feats.extend([[lm.x, lm.y, lm.z] for lm in res_pose.pose_landmarks.landmark])
        if res_hands.multi_hand_landmarks:
            for hand in res_hands.multi_hand_landmarks:
                feats.extend([[lm.x, lm.y, lm.z] for lm in hand.landmark])
        if res_face.multi_face_landmarks:
            for f in res_face.multi_face_landmarks:
                feats.extend([[lm.x, lm.y, lm.z] for lm in f.landmark])

        arr = np.array(feats, dtype=np.float32).flatten()
        flat[:min(flat.size, arr.size)] = arr[:flat.size]
        return flat

    def __getitem__(self, idx):
        video_path = self.videos[idx]
        vid_id = video_path.split("/")[-1].split(".")[0]
        meta_info = video_meta.get(vid_id, None)

        cap = cv2.VideoCapture(video_path)
        frames = []
        f_start = meta_info["frame_start"] if meta_info else 1
        f_end = meta_info["frame_end"] if meta_info and meta_info["frame_end"] > 0 else int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        bbox = meta_info["bbox"] if meta_info else [0, 0, int(cap.get(3)), int(cap.get(4))]

        for i in range(f_start, f_end + 1):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                break
            x1, y1, x2, y2 = bbox
            cropped = frame[y1:y2, x1:x2]
            resized = cv2.resize(cropped, (256, 256))
            frame_rgb = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
            features = self.extract_features(frame_rgb)
            frames.append(features)
            if len(frames) >= self.max_frames:
                break
        cap.release()

        # pad or truncate
        while len(frames) < self.max_frames:
            frames.append(np.zeros((self.feature_dim,), dtype=np.float32))
        if len(frames) > self.max_frames:
            frames = frames[:self.max_frames]

        X = np.stack(frames)
        y = self.labels[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.videos)


In [32]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd

train_losses, test_accs = [], []

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            preds = model(X_test)
            _, predicted = torch.max(preds, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()

    acc = 100 * correct / total
    test_accs.append(acc)
    print(f"✅ Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {acc:.2f}%")

# =======================================================
# ✅ RESULTS
# =======================================================
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(test_accs, label='Accuracy')
plt.legend()
plt.show()

df = pd.DataFrame({
    "epoch": list(range(1, epochs+1)),
    "loss": train_losses,
    "accuracy": test_accs
})
df.to_csv("/kaggle/working/training_log.csv", index=False)
print("\n✅ Training complete! Log saved at /kaggle/working/training_log.csv")


Epoch 1/3:   0%|          | 0/100 [00:00<?, ?it/s]

W0000 00:00:1762609241.950666    2240 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609241.959033    2241 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609241.967111    2236 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609241.996919    2236 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609242.023060    2234 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609242.067434    2234 inference_feedback_manager.cc:114] Feedback manager 

ValueError: all input arrays must have the same shape

In [33]:
import os
import json
import cv2
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image
import mediapipe as mp
import kagglehub


wlasl_path = kagglehub.dataset_download("risangbaskoro/wlasl-processed")

print("Path to dataset files:", wlasl_path)

videos_dir      = os.path.join(wlasl_path, "videos")
json_file       = os.path.join(wlasl_path, "WLASL_v0.3.json")
class_list_file = os.path.join(wlasl_path, "wlasl_class_list.txt")

# Load gloss → index
gloss_to_idx = {}
with open(class_list_file, "r", encoding="utf‑8") as f:
    for idx, line in enumerate(f):
        gloss = line[1:].strip()
        if gloss:
            gloss_to_idx[gloss] = idx
print(f"Loaded {len(gloss_to_idx)} gloss classes.")

# Load JSON annotation
with open(json_file, "r", encoding="utf‑8") as f:
    data = json.load(f)
print(f"Loaded {len(data)} gloss entries.")

# %% [code]
video_paths = []
labels      = []
meta_info   = []

for entry in data:
    gloss = entry.get("gloss")
    if gloss not in gloss_to_idx:
        continue
    class_idx = gloss_to_idx[gloss]
    for inst in entry.get("instances", []):
        vid       = inst.get("video_id")
        fname     = vid + ".mp4"
        fullpath  = os.path.join(videos_dir, fname)
        if not os.path.isfile(fullpath):
            alt = vid.zfill(5) + ".mp4"
            altpath = os.path.join(videos_dir, alt)
            if os.path.isfile(altpath):
                fullpath = altpath
            else:
                continue
        video_paths.append(fullpath)
        labels.append(class_idx)
        meta_info.append({
            "video_id": vid,
            "gloss": gloss,
            "class_idx": class_idx,
            "frame_start": inst.get("frame_start", 0),
            "frame_end":   inst.get("frame_end", None),
            "bbox":        inst.get("bbox", None)
        })

print(f"Collected {len(video_paths)} videos and {len(labels)} labels.")

# Pre‑processing setup
mp_hands  = mp.solutions.hands
hands_det = mp_hands.Hands(static_image_mode=False,
                           max_num_hands=2,
                           min_detection_confidence=0.5,
                           min_tracking_confidence=0.5)

cnn_model = models.resnet18(pretrained=True)
cnn_model.fc = nn.Identity()
cnn_model = cnn_model.eval().cuda()

transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406],
                std=[0.229,0.224,0.225])
])

def extract_frame_features(video_path, max_frames=100, sample_rate=2, bbox=None):
    cap       = cv2.VideoCapture(video_path)
    frames    = []
    idx       = 0
    collected = 0
    while cap.isOpened() and collected < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if idx % sample_rate == 0:
            # full‑frame CNN
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_t   = transform(img_pil).unsqueeze(0).cuda()
            with torch.no_grad():
                feat = cnn_model(img_t).cpu().numpy().reshape(-1)
            # hand keypoints
            if bbox is not None:
                x1,y1,x2,y2 = bbox
                frame_crop = frame[y1:y2, x1:x2]
            else:
                frame_crop = frame
            crop_rgb = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2RGB)
            results  = hands_det.process(crop_rgb)
            if results.multi_hand_landmarks:
                lm  = results.multi_hand_landmarks[0]
                kp  = []
                for l in lm.landmark:
                    kp.extend([l.x, l.y, l.z])
            else:
                kp = [0.0]*63
            feat_and_kp = np.concatenate([feat, kp], axis=0)
            frames.append(feat_and_kp)
            collected += 1
        idx += 1
    cap.release()
    arr = np.array(frames, dtype=np.float32)
    if arr.shape[0] < max_frames:
        pad_amt = max_frames - arr.shape[0]
        pad_arr = np.zeros((pad_amt, arr.shape[1]), dtype=np.float32)
        arr     = np.vstack([arr, pad_arr])
    else:
        arr = arr[:max_frames]
    return arr

# %% [code]
class MultiModalSLDataset(Dataset):
    def __init__(self, paths, labels, meta, max_frames=100, sample_rate=2):
        self.paths       = paths
        self.labels      = labels
        self.meta        = meta
        self.max_frames  = max_frames
        self.sample_rate = sample_rate

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        l = self.labels[idx]
        m = self.meta[idx]
        arr = extract_frame_features(p,
                                     max_frames=self.max_frames,
                                     sample_rate=self.sample_rate,
                                     bbox=m.get("bbox", None))
        return torch.tensor(arr, dtype=torch.float32), torch.tensor(l, dtype=torch.long)

# %% [code]
X_tr, X_val, y_tr, y_val, m_tr, m_val = train_test_split(
    video_paths, labels, meta_info,
    test_size=0.2,
    stratify=labels,
    random_state=42
)

train_ds = MultiModalSLDataset(X_tr, y_tr, m_tr, max_frames=100, sample_rate=2)
val_ds   = MultiModalSLDataset(X_val, y_val, m_val, max_frames=100, sample_rate=2)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0, pin_memory=True)

# %% [code]
class SLRecognitionModel(nn.Module):
    def __init__(self, feat_size, hidden=256, num_layers=2, num_classes=None):
        super().__init__()
        self.lstm = nn.LSTM(feat_size, hidden,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True,
                            dropout=0.5)
        self.fc   = nn.Linear(hidden*2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        last    = out[:, -1, :]
        logits  = self.fc(last)
        return logits

# infer feature size
dummy = extract_frame_features(video_paths[0],
                               max_frames=100, sample_rate=2,
                               bbox=meta_info[0].get("bbox", None))
feat_size = dummy.shape[1]

device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model       = SLRecognitionModel(feat_size=feat_size,
                                 hidden=256,
                                 num_layers=2,
                                 num_classes=len(gloss_to_idx)).to(device)
criterion   = nn.CrossEntropyLoss()
optimizer   = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)

# %% [code]
def train(model, train_loader, val_loader, device, epochs=20):
    best_acc = 0.0
    for e in range(epochs):
        model.train()
        bar = tqdm(train_loader, total=len(train_loader),
                   desc=f"Epoch {e+1}/{epochs}")
        running, correct, total = 0.0, 0, 0
        for X, y in bar:
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            running += loss.item()*X.size(0)
            _, preds = torch.max(logits,1)
            correct   += (preds==y).sum().item()
            total     += y.size(0)
            bar.set_postfix(loss=f"{loss.item():.4f}",
                             acc=f"{correct/total:.4f}")
        epoch_loss = running/total
        epoch_acc  = correct/total
        print(f"\nTrain Loss={epoch_loss:.4f} Acc={epoch_acc:.4f}")

        # validation
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X = X.to(device)
                y = y.to(device)
                logits = model(X)
                _, preds = torch.max(logits,1)
                val_correct += (preds==y).sum().item()
                val_total   += y.size(0)
        val_acc = val_correct/val_total
        print(f"Val Acc={val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_multimodal_model.pth")
            print("Saved best model.\n")
    return model

trained_model = train(model, train_loader, val_loader, device, epochs=20)

# %% [code]
def evaluate_single(model, video_path, bbox=None, max_frames=100, sample_rate=2):
    model.eval()
    arr = extract_frame_features(video_path, max_frames=max_frames,
                                 sample_rate=sample_rate, bbox=bbox)
    inp = torch.tensor(arr[None,:,:], dtype=torch.float32).to(device)
    start = time.time()
    with torch.no_grad():
        logits = model(inp)
        _, pred = torch.max(logits,1)
    end     = time.time()
    latency = end - start
    fps     = 1.0 / latency if latency > 0 else float("inf")
    return latency, fps, pred.item()

sample_video = video_paths[0]
latency, fps, pred_class = evaluate_single(trained_model,
                                           sample_video,
                                           bbox=meta_info[0].get("bbox", None),
                                           max_frames=100,
                                           sample_rate=2)
print(f"Inference Time: {latency:.4f} s, FPS: {fps:.2f}, Predicted Class Index: {pred_class}")

Path to dataset files: /kaggle/input/wlasl-processed
Loaded 2000 gloss classes.
Loaded 2000 gloss entries.
Collected 119 videos and 119 labels.


W0000 00:00:1762609554.920215    2477 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609554.945321    2477 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 94.1MB/s]
Unable to load any of {libcudnn_engines_precompiled.so.9.3.0, libcudnn_engines_precompiled.so.9.3, libcudnn_engines_precompiled.so.9, libcudnn_engines_precompiled.so}


RuntimeError: ptrDesc->finalize()

In [34]:
Path to dataset files: /kaggle/input/wlasl-processed
Loaded 2000 gloss classes.
Loaded 2000 gloss entries.
Collected 119 videos and 119 labels.
W0000 00:00:1762609554.920215    2477 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609554.945321    2477 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 94.1MB/s]
Unable to load any of {libcudnn_engines_precompiled.so.9.3.0, libcudnn_engines_precompiled.so.9.3, libcudnn_engines_precompiled.so.9, libcudnn_engines_precompiled.so}
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
/tmp/ipykernel_48/52941209.py in <cell line: 0>()
    190 
    191 # infer feature size
--> 192 dummy = extract_frame_features(video_paths[0],
    193                                max_frames=100, sample_rate=2,
    194                                bbox=meta_info[0].get("bbox", None))

/tmp/ipykernel_48/52941209.py in extract_frame_features(video_path, max_frames, sample_rate, bbox)
    106             img_t   = transform(img_pil).unsqueeze(0).cuda()
    107             with torch.no_grad():
--> 108                 feat = cnn_model(img_t).cpu().numpy().reshape(-1)
    109             # hand keypoints
    110             if bbox is not None:

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py in _wrapped_call_impl(self, *args, **kwargs)
   1737             return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1738         else:
-> 1739             return self._call_impl(*args, **kwargs)
   1740 
   1741     # torchrec tests the code consistency with the following code

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py in _call_impl(self, *args, **kwargs)
   1748                 or _global_backward_pre_hooks or _global_backward_hooks
   1749                 or _global_forward_hooks or _global_forward_pre_hooks):
-> 1750             return forward_call(*args, **kwargs)
   1751 
   1752         result = None

/usr/local/lib/python3.11/dist-packages/torchvision/models/resnet.py in forward(self, x)
    283 
    284     def forward(self, x: Tensor) -> Tensor:
--> 285         return self._forward_impl(x)
    286 
    287 

/usr/local/lib/python3.11/dist-packages/torchvision/models/resnet.py in _forward_impl(self, x)
    266     def _forward_impl(self, x: Tensor) -> Tensor:
    267         # See note [TorchScript super()]
--> 268         x = self.conv1(x)
    269         x = self.bn1(x)
    270         x = self.relu(x)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py in _wrapped_call_impl(self, *args, **kwargs)
   1737             return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1738         else:
-> 1739             return self._call_impl(*args, **kwargs)
   1740 
   1741     # torchrec tests the code consistency with the following code

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py in _call_impl(self, *args, **kwargs)
   1748                 or _global_backward_pre_hooks or _global_backward_hooks
   1749                 or _global_forward_hooks or _global_forward_pre_hooks):
-> 1750             return forward_call(*args, **kwargs)
   1751 
   1752         result = None

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py in forward(self, input)
    552 
    553     def forward(self, input: Tensor) -> Tensor:
--> 554         return self._conv_forward(input, self.weight, self.bias)
    555 
    556 

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py in _conv_forward(self, input, weight, bias)
    547                 self.groups,
    548             )
--> 549         return F.conv2d(
    550             input, weight, bias, self.stride, self.padding, self.dilation, self.groups
    551         )

RuntimeError: ptrDesc->finalize()

SyntaxError: unmatched ']' (1662505788.py, line 5)

In [35]:
# ============================================================
# ✅ CNN + Mediapipe Feature Extractor Setup (Fixed)
# ============================================================

import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as T
import mediapipe as mp
import cv2
import numpy as np
from PIL import Image

# ---- cuDNN safe configuration ----
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

# ---- Mediapipe setup ----
mp_hands  = mp.solutions.hands
hands_det = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# ---- ResNet18 Feature Extractor ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
cnn_model.fc = nn.Identity()
cnn_model = cnn_model.to(device).eval()

print(f"✅ CNN initialized on: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")

# ---- Image transform ----
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# ============================================================
# ✅ Frame-Level Feature Extraction Function
# ============================================================

def extract_frame_features(video_path, max_frames=100, sample_rate=2, bbox=None):
    cap       = cv2.VideoCapture(video_path)
    frames    = []
    idx       = 0
    collected = 0

    while cap.isOpened() and collected < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if idx % sample_rate == 0:
            # --- CNN visual feature ---
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_t   = transform(img_pil).unsqueeze(0).to(device)

            with torch.no_grad():
                feat = cnn_model(img_t).detach().cpu().numpy().reshape(-1)

            # --- Hand keypoints (Mediapipe) ---
            if bbox is not None:
                x1, y1, x2, y2 = bbox
                frame_crop = frame[y1:y2, x1:x2]
            else:
                frame_crop = frame

            crop_rgb = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2RGB)
            results  = hands_det.process(crop_rgb)

            if results.multi_hand_landmarks:
                lm  = results.multi_hand_landmarks[0]
                kp  = []
                for l in lm.landmark:
                    kp.extend([l.x, l.y, l.z])
            else:
                kp = [0.0] * 63  # fallback for missing hands

            # --- Combine CNN + hand features ---
            feat_and_kp = np.concatenate([feat, kp], axis=0)
            frames.append(feat_and_kp)
            collected += 1
        idx += 1

    cap.release()
    arr = np.array(frames, dtype=np.float32)

    # --- Pad or truncate to fixed length ---
    if arr.shape[0] < max_frames:
        pad_amt = max_frames - arr.shape[0]
        pad_arr = np.zeros((pad_amt, arr.shape[1]), dtype=np.float32)
        arr     = np.vstack([arr, pad_arr])
    else:
        arr = arr[:max_frames]

    return arr


✅ CNN initialized on: cuda (Tesla T4)


W0000 00:00:1762609673.629655    2487 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609673.640450    2487 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [36]:
# ============================================================
# 🔹 Imports & Setup
# ============================================================
import os
import json
import cv2
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image
import mediapipe as mp
import kagglehub
import torch.backends.cudnn as cudnn

# ============================================================
# 🔹 GPU / cuDNN Safe Initialization
# ============================================================
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ============================================================
# 🔹 Dataset Loading
# ============================================================
wlasl_path = kagglehub.dataset_download("risangbaskoro/wlasl-processed")
print("Path to dataset files:", wlasl_path)

videos_dir      = os.path.join(wlasl_path, "videos")
json_file       = os.path.join(wlasl_path, "WLASL_v0.3.json")
class_list_file = os.path.join(wlasl_path, "wlasl_class_list.txt")

# Load gloss → index
gloss_to_idx = {}
with open(class_list_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        gloss = line[1:].strip()
        if gloss:
            gloss_to_idx[gloss] = idx
print(f"Loaded {len(gloss_to_idx)} gloss classes.")

# Load JSON annotation
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"Loaded {len(data)} gloss entries.")

# Collect video paths and metadata
video_paths, labels, meta_info = [], [], []
for entry in data:
    gloss = entry.get("gloss")
    if gloss not in gloss_to_idx:
        continue
    class_idx = gloss_to_idx[gloss]
    for inst in entry.get("instances", []):
        vid = inst.get("video_id")
        fname = vid + ".mp4"
        fullpath = os.path.join(videos_dir, fname)
        if not os.path.isfile(fullpath):
            alt = vid.zfill(5) + ".mp4"
            altpath = os.path.join(videos_dir, alt)
            if os.path.isfile(altpath):
                fullpath = altpath
            else:
                continue
        video_paths.append(fullpath)
        labels.append(class_idx)
        meta_info.append({
            "video_id": vid,
            "gloss": gloss,
            "class_idx": class_idx,
            "frame_start": inst.get("frame_start", 0),
            "frame_end": inst.get("frame_end", None),
            "bbox": inst.get("bbox", None)
        })

print(f"✅ Collected {len(video_paths)} videos and {len(labels)} labels.")

# ============================================================
# 🔹 Mediapipe + ResNet18 Setup (Fixed)
# ============================================================
mp_hands  = mp.solutions.hands
hands_det = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
cnn_model.fc = nn.Identity()
cnn_model = cnn_model.to(device).eval()
print(f"✅ CNN initialized on: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# ============================================================
# 🔹 Feature Extraction Function
# ============================================================
def extract_frame_features(video_path, max_frames=100, sample_rate=2, bbox=None):
    cap = cv2.VideoCapture(video_path)
    frames, idx, collected = [], 0, 0

    while cap.isOpened() and collected < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if idx % sample_rate == 0:
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_t   = transform(img_pil).unsqueeze(0).to(device)
            with torch.no_grad():
                feat = cnn_model(img_t).detach().cpu().numpy().reshape(-1)
            if bbox is not None:
                x1, y1, x2, y2 = bbox
                frame_crop = frame[y1:y2, x1:x2]
            else:
                frame_crop = frame
            crop_rgb = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2RGB)
            results = hands_det.process(crop_rgb)
            if results.multi_hand_landmarks:
                lm  = results.multi_hand_landmarks[0]
                kp  = []
                for l in lm.landmark:
                    kp.extend([l.x, l.y, l.z])
            else:
                kp = [0.0] * 63
            feat_and_kp = np.concatenate([feat, kp], axis=0)
            frames.append(feat_and_kp)
            collected += 1
        idx += 1
    cap.release()

    arr = np.array(frames, dtype=np.float32)
    if arr.shape[0] < max_frames:
        pad_amt = max_frames - arr.shape[0]
        pad_arr = np.zeros((pad_amt, arr.shape[1]), dtype=np.float32)
        arr = np.vstack([arr, pad_arr])
    else:
        arr = arr[:max_frames]
    return arr

# ============================================================
# 🔹 Dataset Definition
# ============================================================
class MultiModalSLDataset(Dataset):
    def __init__(self, paths, labels, meta, max_frames=100, sample_rate=2):
        self.paths = paths
        self.labels = labels
        self.meta = meta
        self.max_frames = max_frames
        self.sample_rate = sample_rate

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        l = self.labels[idx]
        m = self.meta[idx]
        arr = extract_frame_features(
            p,
            max_frames=self.max_frames,
            sample_rate=self.sample_rate,
            bbox=m.get("bbox", None)
        )
        return torch.tensor(arr, dtype=torch.float32), torch.tensor(l, dtype=torch.long)

# ============================================================
# 🔹 Split Data
# ============================================================
X_tr, X_val, y_tr, y_val, m_tr, m_val = train_test_split(
    video_paths, labels, meta_info, test_size=0.2, stratify=labels, random_state=42
)

train_ds = MultiModalSLDataset(X_tr, y_tr, m_tr, max_frames=100, sample_rate=2)
val_ds   = MultiModalSLDataset(X_val, y_val, m_val, max_frames=100, sample_rate=2)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=0)

# ============================================================
# 🔹 Model Definition
# ============================================================
class SLRecognitionModel(nn.Module):
    def __init__(self, feat_size, hidden=256, num_layers=2, num_classes=None):
        super().__init__()
        self.lstm = nn.LSTM(feat_size, hidden,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True,
                            dropout=0.5)
        self.fc = nn.Linear(hidden * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)

# ============================================================
# 🔹 Initialize Model
# ============================================================
dummy = extract_frame_features(video_paths[0],
                               max_frames=100, sample_rate=2,
                               bbox=meta_info[0].get("bbox", None))
feat_size = dummy.shape[1]

model = SLRecognitionModel(feat_size=feat_size,
                           hidden=256,
                           num_layers=2,
                           num_classes=len(gloss_to_idx)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)

# ============================================================
# 🔹 Training Loop
# ============================================================
def train(model, train_loader, val_loader, device, epochs=5):
    best_acc = 0.0
    for e in range(epochs):
        model.train()
        bar = tqdm(train_loader, total=len(train_loader), desc=f"Epoch {e+1}/{epochs}")
        running, correct, total = 0.0, 0, 0
        for X, y in bar:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            running += loss.item() * X.size(0)
            _, preds = torch.max(logits, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            bar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{correct/total:.4f}")
        epoch_loss = running / total
        epoch_acc = correct / total
        print(f"\nTrain Loss={epoch_loss:.4f} Acc={epoch_acc:.4f}")

        # validation
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                logits = model(X)
                _, preds = torch.max(logits, 1)
                val_correct += (preds == y).sum().item()
                val_total += y.size(0)
        val_acc = val_correct / val_total
        print(f"Val Acc={val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_multimodal_model.pth")
            print("✅ Saved best model.\n")
    return model

# ============================================================
# 🔹 Run Training
# ============================================================
trained_model = train(model, train_loader, val_loader, device, epochs=5)

# ============================================================
# 🔹 Evaluate a Sample
# ============================================================
def evaluate_single(model, video_path, bbox=None, max_frames=100, sample_rate=2):
    model.eval()
    arr = extract_frame_features(video_path, max_frames=max_frames, sample_rate=sample_rate, bbox=bbox)
    inp = torch.tensor(arr[None, :, :], dtype=torch.float32).to(device)
    start = time.time()
    with torch.no_grad():
        logits = model(inp)
        _, pred = torch.max(logits, 1)
    end = time.time()
    latency = end - start
    fps = 1.0 / latency if latency > 0 else float("inf")
    return latency, fps, pred.item()

sample_video = video_paths[0]
latency, fps, pred_class = evaluate_single(trained_model,
                                           sample_video,
                                           bbox=meta_info[0].get("bbox", None),
                                           max_frames=100,
                                           sample_rate=2)
print(f"Inference Time: {latency:.4f}s | FPS: {fps:.2f} | Predicted Class Index: {pred_class}")


✅ Using device: cuda
GPU: Tesla T4
Path to dataset files: /kaggle/input/wlasl-processed
Loaded 2000 gloss classes.
Loaded 2000 gloss entries.
✅ Collected 119 videos and 119 labels.


W0000 00:00:1762609732.136648    2493 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609732.152964    2493 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ CNN initialized on: cuda (Tesla T4)


RuntimeError: ptrDesc->finalize()

In [37]:
# ============================================================
# ✅ cuDNN Safety Patch for Kaggle / Tesla T4
# ============================================================
import os
import torch

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["NVIDIA_TF32_OVERRIDE"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# disable TensorFlow / Mediapipe cuDNN pre-init interference
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.enabled = True

print("⚙️  Applied cuDNN safety patch (deterministic mode)")


⚙️  Applied cuDNN safety patch (deterministic mode)


In [38]:
# ============================================================
# ✅ ResNet18 (Safe GPU Load)
# ============================================================
from torchvision import models
import torch.nn as nn

try:
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
except Exception as e:
    print("⚠️ Weight load failed, loading without pretrained weights:", e)
    cnn_model = models.resnet18(weights=None)

cnn_model.fc = nn.Identity()

# move to CUDA safely
cnn_model = cnn_model.to(device)
torch.cuda.synchronize()
cnn_model.eval()

print(f"✅ CNN initialized safely on: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")


✅ CNN initialized safely on: cuda (Tesla T4)


In [39]:
import os
import json
import cv2
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image
import mediapipe as mp
import kagglehub


wlasl_path = kagglehub.dataset_download("risangbaskoro/wlasl-processed")

print("Path to dataset files:", wlasl_path)

videos_dir      = os.path.join(wlasl_path, "videos")
json_file       = os.path.join(wlasl_path, "WLASL_v0.3.json")
class_list_file = os.path.join(wlasl_path, "wlasl_class_list.txt")

# Load gloss → index
gloss_to_idx = {}
with open(class_list_file, "r", encoding="utf‑8") as f:
    for idx, line in enumerate(f):
        gloss = line[1:].strip()
        if gloss:
            gloss_to_idx[gloss] = idx
print(f"Loaded {len(gloss_to_idx)} gloss classes.")

# Load JSON annotation
with open(json_file, "r", encoding="utf‑8") as f:
    data = json.load(f)
print(f"Loaded {len(data)} gloss entries.")

# %% [code]
video_paths = []
labels      = []
meta_info   = []

for entry in data:
    gloss = entry.get("gloss")
    if gloss not in gloss_to_idx:
        continue
    class_idx = gloss_to_idx[gloss]
    for inst in entry.get("instances", []):
        vid       = inst.get("video_id")
        fname     = vid + ".mp4"
        fullpath  = os.path.join(videos_dir, fname)
        if not os.path.isfile(fullpath):
            alt = vid.zfill(5) + ".mp4"
            altpath = os.path.join(videos_dir, alt)
            if os.path.isfile(altpath):
                fullpath = altpath
            else:
                continue
        video_paths.append(fullpath)
        labels.append(class_idx)
        meta_info.append({
            "video_id": vid,
            "gloss": gloss,
            "class_idx": class_idx,
            "frame_start": inst.get("frame_start", 0),
            "frame_end":   inst.get("frame_end", None),
            "bbox":        inst.get("bbox", None)
        })

print(f"Collected {len(video_paths)} videos and {len(labels)} labels.")

# Pre‑processing setup
mp_hands  = mp.solutions.hands
hands_det = mp_hands.Hands(static_image_mode=False,
                           max_num_hands=2,
                           min_detection_confidence=0.5,
                           min_tracking_confidence=0.5)

cnn_model = models.resnet18(pretrained=True)
cnn_model.fc = nn.Identity()
cnn_model = cnn_model.eval().cuda()

transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406],
                std=[0.229,0.224,0.225])
])

def extract_frame_features(video_path, max_frames=100, sample_rate=2, bbox=None):
    cap       = cv2.VideoCapture(video_path)
    frames    = []
    idx       = 0
    collected = 0
    while cap.isOpened() and collected < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if idx % sample_rate == 0:
            # full‑frame CNN
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_t   = transform(img_pil).unsqueeze(0).cuda()
            with torch.no_grad():
                feat = cnn_model(img_t).cpu().numpy().reshape(-1)
            # hand keypoints
            if bbox is not None:
                x1,y1,x2,y2 = bbox
                frame_crop = frame[y1:y2, x1:x2]
            else:
                frame_crop = frame
            crop_rgb = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2RGB)
            results  = hands_det.process(crop_rgb)
            if results.multi_hand_landmarks:
                lm  = results.multi_hand_landmarks[0]
                kp  = []
                for l in lm.landmark:
                    kp.extend([l.x, l.y, l.z])
            else:
                kp = [0.0]*63
            feat_and_kp = np.concatenate([feat, kp], axis=0)
            frames.append(feat_and_kp)
            collected += 1
        idx += 1
    cap.release()
    arr = np.array(frames, dtype=np.float32)
    if arr.shape[0] < max_frames:
        pad_amt = max_frames - arr.shape[0]
        pad_arr = np.zeros((pad_amt, arr.shape[1]), dtype=np.float32)
        arr     = np.vstack([arr, pad_arr])
    else:
        arr = arr[:max_frames]
    return arr

# %% [code]
class MultiModalSLDataset(Dataset):
    def __init__(self, paths, labels, meta, max_frames=100, sample_rate=2):
        self.paths       = paths
        self.labels      = labels
        self.meta        = meta
        self.max_frames  = max_frames
        self.sample_rate = sample_rate

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        l = self.labels[idx]
        m = self.meta[idx]
        arr = extract_frame_features(p,
                                     max_frames=self.max_frames,
                                     sample_rate=self.sample_rate,
                                     bbox=m.get("bbox", None))
        return torch.tensor(arr, dtype=torch.float32), torch.tensor(l, dtype=torch.long)

# %% [code]
X_tr, X_val, y_tr, y_val, m_tr, m_val = train_test_split(
    video_paths, labels, meta_info,
    test_size=0.2,
    stratify=labels,
    random_state=42
)

train_ds = MultiModalSLDataset(X_tr, y_tr, m_tr, max_frames=100, sample_rate=2)
val_ds   = MultiModalSLDataset(X_val, y_val, m_val, max_frames=100, sample_rate=2)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0, pin_memory=True)

# %% [code]
class SLRecognitionModel(nn.Module):
    def __init__(self, feat_size, hidden=256, num_layers=2, num_classes=None):
        super().__init__()
        self.lstm = nn.LSTM(feat_size, hidden,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True,
                            dropout=0.5)
        self.fc   = nn.Linear(hidden*2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        last    = out[:, -1, :]
        logits  = self.fc(last)
        return logits

# infer feature size
dummy = extract_frame_features(video_paths[0],
                               max_frames=100, sample_rate=2,
                               bbox=meta_info[0].get("bbox", None))
feat_size = dummy.shape[1]

device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model       = SLRecognitionModel(feat_size=feat_size,
                                 hidden=256,
                                 num_layers=2,
                                 num_classes=len(gloss_to_idx)).to(device)
criterion   = nn.CrossEntropyLoss()
optimizer   = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)

# %% [code]
def train(model, train_loader, val_loader, device, epochs=20):
    best_acc = 0.0
    for e in range(epochs):
        model.train()
        bar = tqdm(train_loader, total=len(train_loader),
                   desc=f"Epoch {e+1}/{epochs}")
        running, correct, total = 0.0, 0, 0
        for X, y in bar:
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            running += loss.item()*X.size(0)
            _, preds = torch.max(logits,1)
            correct   += (preds==y).sum().item()
            total     += y.size(0)
            bar.set_postfix(loss=f"{loss.item():.4f}",
                             acc=f"{correct/total:.4f}")
        epoch_loss = running/total
        epoch_acc  = correct/total
        print(f"\nTrain Loss={epoch_loss:.4f} Acc={epoch_acc:.4f}")

        # validation
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X = X.to(device)
                y = y.to(device)
                logits = model(X)
                _, preds = torch.max(logits,1)
                val_correct += (preds==y).sum().item()
                val_total   += y.size(0)
        val_acc = val_correct/val_total
        print(f"Val Acc={val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_multimodal_model.pth")
            print("Saved best model.\n")
    return model

trained_model = train(model, train_loader, val_loader, device, epochs=20)

# %% [code]
def evaluate_single(model, video_path, bbox=None, max_frames=100, sample_rate=2):
    model.eval()
    arr = extract_frame_features(video_path, max_frames=max_frames,
                                 sample_rate=sample_rate, bbox=bbox)
    inp = torch.tensor(arr[None,:,:], dtype=torch.float32).to(device)
    start = time.time()
    with torch.no_grad():
        logits = model(inp)
        _, pred = torch.max(logits,1)
    end     = time.time()
    latency = end - start
    fps     = 1.0 / latency if latency > 0 else float("inf")
    return latency, fps, pred.item()

sample_video = video_paths[0]
latency, fps, pred_class = evaluate_single(trained_model,
                                           sample_video,
                                           bbox=meta_info[0].get("bbox", None),
                                           max_frames=100,
                                           sample_rate=2)
print(f"Inference Time: {latency:.4f} s, FPS: {fps:.2f}, Predicted Class Index: {pred_class}")

Path to dataset files: /kaggle/input/wlasl-processed
Loaded 2000 gloss classes.
Loaded 2000 gloss entries.
Collected 119 videos and 119 labels.


W0000 00:00:1762609790.874786    2505 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609790.886804    2505 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


RuntimeError: ptrDesc->finalize()

In [40]:
# ============================================================
# ✅ CNN Initialization (Safe & GPU-Stable)
# ============================================================
from torchvision import models
import torch.nn as nn
import torchvision.transforms as T
import mediapipe as mp
from PIL import Image
import cv2
import numpy as np

# Mediapipe setup
mp_hands = mp.solutions.hands
hands_det = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# ResNet18 feature extractor
try:
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
except Exception as e:
    print("⚠️ Could not load pretrained weights, using random init:", e)
    cnn_model = models.resnet18(weights=None)

cnn_model.fc = nn.Identity()
cnn_model = cnn_model.to(device).eval()
torch.cuda.synchronize()

print(f"✅ CNN ready on {device}: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# Image transformation
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# ============================================================
# ✅ Frame-Level Feature Extraction Function
# ============================================================
def extract_frame_features(video_path, max_frames=100, sample_rate=2, bbox=None):
    cap = cv2.VideoCapture(video_path)
    frames, idx, collected = [], 0, 0

    while cap.isOpened() and collected < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if idx % sample_rate == 0:
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_t = transform(img_pil).unsqueeze(0).to(device)
            with torch.no_grad():
                feat = cnn_model(img_t).detach().cpu().numpy().reshape(-1)

            # hand keypoints
            frame_crop = frame
            if bbox is not None and len(bbox) == 4:
                x1, y1, x2, y2 = map(int, bbox)
                frame_crop = frame[y1:y2, x1:x2]

            crop_rgb = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2RGB)
            results = hands_det.process(crop_rgb)
            if results.multi_hand_landmarks:
                lm = results.multi_hand_landmarks[0]
                kp = []
                for l in lm.landmark:
                    kp.extend([l.x, l.y, l.z])
            else:
                kp = [0.0] * 63

            feat_and_kp = np.concatenate([feat, kp], axis=0)
            frames.append(feat_and_kp)
            collected += 1
        idx += 1

    cap.release()
    if len(frames) == 0:
        # fallback if empty video
        feat_dim = 512 + 63
        return np.zeros((max_frames, feat_dim), dtype=np.float32)

    arr = np.array(frames, dtype=np.float32)
    if arr.shape[0] < max_frames:
        pad_amt = max_frames - arr.shape[0]
        pad_arr = np.zeros((pad_amt, arr.shape[1]), dtype=np.float32)
        arr = np.vstack([arr, pad_arr])
    else:
        arr = arr[:max_frames]

    return arr

# ============================================================
# ✅ Dataset Class
# ============================================================
from torch.utils.data import Dataset, DataLoader
class MultiModalSLDataset(Dataset):
    def __init__(self, paths, labels, meta, max_frames=100, sample_rate=2):
        self.paths = paths
        self.labels = labels
        self.meta = meta
        self.max_frames = max_frames
        self.sample_rate = sample_rate

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        l = self.labels[idx]
        m = self.meta[idx]
        arr = extract_frame_features(
            p,
            max_frames=self.max_frames,
            sample_rate=self.sample_rate,
            bbox=m.get("bbox", None)
        )
        return torch.tensor(arr, dtype=torch.float32), torch.tensor(l, dtype=torch.long)

# ============================================================
# ✅ Data Split & Loaders
# ============================================================
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val, m_tr, m_val = train_test_split(
    video_paths, labels, meta_info,
    test_size=0.2, stratify=labels, random_state=42
)

train_ds = MultiModalSLDataset(X_tr, y_tr, m_tr)
val_ds = MultiModalSLDataset(X_val, y_val, m_val)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=0)

# ============================================================
# ✅ Model Definition
# ============================================================
class SLRecognitionModel(nn.Module):
    def __init__(self, feat_size, hidden=256, num_layers=2, num_classes=None):
        super().__init__()
        self.lstm = nn.LSTM(
            feat_size, hidden,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.5
        )
        self.fc = nn.Linear(hidden * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)

# Infer feature size safely
dummy = extract_frame_features(video_paths[0], max_frames=10, sample_rate=2,
                               bbox=meta_info[0].get("bbox", None))
feat_size = dummy.shape[1]
print("✅ Feature size:", feat_size)

model = SLRecognitionModel(
    feat_size=feat_size,
    hidden=256,
    num_layers=2,
    num_classes=len(gloss_to_idx)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# ============================================================
# ✅ Training Loop
# ============================================================
from tqdm import tqdm

def train(model, train_loader, val_loader, device, epochs=3):
    best_acc = 0.0
    for e in range(epochs):
        model.train()
        total_loss, correct, total = 0.0, 0, 0
        loop = tqdm(train_loader, desc=f"Epoch {e+1}/{epochs}", leave=True)
        for X, y in loop:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * X.size(0)
            _, preds = torch.max(logits, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            loop.set_postfix(loss=f"{loss.item():.4f}", acc=f"{correct/total:.4f}")

        epoch_loss = total_loss / total
        epoch_acc = correct / total
        print(f"\nTrain Loss={epoch_loss:.4f} | Acc={epoch_acc:.4f}")

        # validation
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                logits = model(X)
                _, preds = torch.max(logits, 1)
                val_correct += (preds == y).sum().item()
                val_total += y.size(0)
        val_acc = val_correct / val_total
        print(f"Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_multimodal_model.pth")
            print("✅ Saved best model!\n")

# ============================================================
# ✅ Run Training
# ============================================================
trained_model = train(model, train_loader, val_loader, device, epochs=3)


W0000 00:00:1762609833.157551    2515 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762609833.187009    2515 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ CNN ready on cuda: Tesla T4


RuntimeError: ptrDesc->finalize()

In [41]:
# === Run this whole cell to initialize CNN safely and set cnn_device ===
import os, traceback
import torch, torch.nn as nn
import torch.backends.cudnn as cudnn
from torchvision import models
import torchvision.transforms as T
import mediapipe as mp
from PIL import Image
import numpy as np
import cv2

# ---------- 1) cuDNN / env safety (recommended) ----------
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"   # helps with deterministic error traces
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.allow_tf32 = False
torch.backends.cuda.matmul.allow_tf32 = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device (main):", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# ---------- 2) Mediapipe hands (keep as before) ----------
mp_hands = mp.solutions.hands
hands_det = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# ---------- helper: safe forward test ----------
def try_prewarm_conv(device):
    try:
        # small conv net to pre-initialize cuDNN descriptors / contexts
        small = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU()
        ).to(device)
        small.eval()
        dummy = torch.randn(2, 3, 224, 224, device=device)
        with torch.no_grad():
            out = small(dummy)
        if device.type == "cuda":
            torch.cuda.synchronize()
        return True, None
    except Exception as e:
        tb = traceback.format_exc()
        return False, tb

# ---------- 3) Attempt GPU path with prewarm ----------
cnn_device = device  # aim for GPU if available
use_cpu_fallback = False
if device.type == "cuda":
    ok, tb = try_prewarm_conv(device)
    if ok:
        print("✅ Prewarm conv passed on CUDA.")
        # Now attempt to load ResNet on GPU
        try:
            cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
            cnn_model.fc = nn.Identity()
            cnn_model = cnn_model.to(cnn_device).eval()
            # quick forward test
            test_in = torch.randn(1, 3, 224, 224, device=cnn_device)
            with torch.no_grad():
                _ = cnn_model(test_in)
            if cnn_device.type == "cuda":
                torch.cuda.synchronize()
            print("✅ ResNet loaded & forward OK on GPU.")
        except Exception as e:
            print("⚠️ ResNet on GPU failed. Falling back to CPU for CNN feature extractor.")
            print("Error:\n", e)
            use_cpu_fallback = True
    else:
        print("⚠️ Prewarm conv failed on CUDA. Traceback:\n", tb)
        use_cpu_fallback = True
else:
    print("⚠️ CUDA not available — using CPU for CNN.")

# ---------- 4) CPU fallback if needed ----------
if use_cpu_fallback or device.type != "cuda":
    cnn_device = torch.device("cpu")
    print("➡️ Initializing ResNet on CPU (cnn_device=cpu). This avoids cuDNN crashes.")
    try:
        cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    except Exception as e:
        print("⚠️ Pretrained weights load failed on CPU, using random init.")
        cnn_model = models.resnet18(weights=None)
    cnn_model.fc = nn.Identity()
    cnn_model = cnn_model.to(cnn_device).eval()
    print("✅ ResNet ready on CPU.")

# ---------- 5) Setup transform and final confirmation ----------
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

print(f"Final cnn_device: {cnn_device} (cnn_model on {next(cnn_model.parameters()).device})")

# ---------- 6) Provide an extract_frame_features that uses cnn_device ----------
def extract_frame_features(video_path, max_frames=100, sample_rate=2, bbox=None):
    """Returns numpy array of shape (max_frames, feat_dim) where feat_dim = cnn_feat + 63 hand kps"""
    cap = cv2.VideoCapture(video_path)
    frames, idx, collected = [], 0, 0
    feat_dim = None

    while cap.isOpened() and collected < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if idx % sample_rate == 0:
            # CNN feature (respect cnn_device)
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_t = transform(img_pil).unsqueeze(0).to(cnn_device)
            with torch.no_grad():
                feat = cnn_model(img_t).detach().cpu().numpy().reshape(-1)  # always moved to cpu for numpy
            # hand keypoints (mediapipe uses CPU images; convert RGB)
            if bbox is not None and len(bbox) == 4:
                x1,y1,x2,y2 = map(int, bbox)
                # clamp coords
                h,w = frame.shape[:2]
                x1, x2 = max(0,min(x1,w-1)), max(0,min(x2,w))
                y1, y2 = max(0,min(y1,h-1)), max(0,min(y2,h))
                frame_crop = frame[y1:y2, x1:x2] if (y2>y1 and x2>x1) else frame
            else:
                frame_crop = frame
            crop_rgb = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2RGB)
            results = hands_det.process(crop_rgb)
            if results.multi_hand_landmarks:
                lm = results.multi_hand_landmarks[0]
                kp = []
                for l in lm.landmark:
                    kp.extend([l.x, l.y, l.z])
            else:
                kp = [0.0] * 63
            feat_and_kp = np.concatenate([feat, np.array(kp, dtype=np.float32)], axis=0)
            frames.append(feat_and_kp)
            collected += 1
        idx += 1

    cap.release()
    if len(frames) == 0:
        # fallback sized zeros (determine feat_dim from model output if possible)
        try:
            dummy_in = torch.randn(1,3,224,224, device=cnn_device)
            with torch.no_grad():
                dummy_feat = cnn_model(dummy_in).detach().cpu().numpy().reshape(-1)
            feat_dim = dummy_feat.shape[0] + 63
        except Exception:
            feat_dim = 512 + 63  # resnet18 default fc-out is 512
        return np.zeros((max_frames, feat_dim), dtype=np.float32)

    arr = np.array(frames, dtype=np.float32)
    if arr.shape[0] < max_frames:
        pad_amt = max_frames - arr.shape[0]
        pad_arr = np.zeros((pad_amt, arr.shape[1]), dtype=np.float32)
        arr = np.vstack([arr, pad_arr])
    else:
        arr = arr[:max_frames]
    return arr

# ---------- 7) quick self-test (safe) ----------
try:
    test_path = video_paths[0]  # your existing list from earlier cell
    print("Running quick feature extraction test on:", test_path)
    sample = extract_frame_features(test_path, max_frames=4, sample_rate=2, bbox=meta_info[0].get("bbox", None))
    print("Sample features shape:", sample.shape)
    print("All set — continue training. cnn_device =", cnn_device)
except Exception as e:
    print("❌ Test failed. Full traceback:\n")
    import traceback as _tb
    _tb.print_exc()


Device (main): cuda
GPU: Tesla T4
⚠️ Prewarm conv failed on CUDA. Traceback:
 Traceback (most recent call last):
  File "/tmp/ipykernel_48/1122974886.py", line 48, in try_prewarm_conv
    out = small(dummy)
          ^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/container.py", line 250, in forward
    input = module(input)
            ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dis

W0000 00:00:1762610005.509201    2523 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762610005.520836    2524 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Sample features shape: (4, 575)
All set — continue training. cnn_device = cpu


In [42]:
# ============================================================
# ✅ Data Split & Loaders
# ============================================================
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class MultiModalSLDataset(Dataset):
    def __init__(self, paths, labels, meta, max_frames=100, sample_rate=2):
        self.paths = paths
        self.labels = labels
        self.meta = meta
        self.max_frames = max_frames
        self.sample_rate = sample_rate

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        l = self.labels[idx]
        m = self.meta[idx]
        arr = extract_frame_features(
            p,
            max_frames=self.max_frames,
            sample_rate=self.sample_rate,
            bbox=m.get("bbox", None)
        )
        return torch.tensor(arr, dtype=torch.float32), torch.tensor(l, dtype=torch.long)

# Split data
X_tr, X_val, y_tr, y_val, m_tr, m_val = train_test_split(
    video_paths, labels, meta_info,
    test_size=0.2, stratify=labels, random_state=42
)

train_ds = MultiModalSLDataset(X_tr, y_tr, m_tr)
val_ds   = MultiModalSLDataset(X_val, y_val, m_val)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=0)

# ============================================================
# ✅ LSTM Model
# ============================================================
class SLRecognitionModel(nn.Module):
    def __init__(self, feat_size, hidden=256, num_layers=2, num_classes=None):
        super().__init__()
        self.lstm = nn.LSTM(
            feat_size, hidden,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.5
        )
        self.fc = nn.Linear(hidden * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)

# Infer feature size
dummy = extract_frame_features(video_paths[0], max_frames=10, sample_rate=2, bbox=meta_info[0].get("bbox", None))
feat_size = dummy.shape[1]
print(f"✅ Feature size detected: {feat_size}")

# Model setup
model = SLRecognitionModel(
    feat_size=feat_size,
    hidden=256,
    num_layers=2,
    num_classes=len(gloss_to_idx)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# ============================================================
# ✅ Training Loop
# ============================================================
def train(model, train_loader, val_loader, device, epochs=3):
    best_acc = 0.0
    for e in range(epochs):
        model.train()
        total_loss, correct, total = 0.0, 0, 0
        loop = tqdm(train_loader, desc=f"Epoch {e+1}/{epochs}", leave=True)
        for X, y in loop:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * X.size(0)
            _, preds = torch.max(logits, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            loop.set_postfix(loss=f"{loss.item():.4f}", acc=f"{correct/total:.4f}")

        epoch_loss = total_loss / total
        epoch_acc = correct / total
        print(f"\nTrain Loss={epoch_loss:.4f} | Acc={epoch_acc:.4f}")

        # validation
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                logits = model(X)
                _, preds = torch.max(logits, 1)
                val_correct += (preds == y).sum().item()
                val_total += y.size(0)
        val_acc = val_correct / val_total
        print(f"Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_multimodal_model.pth")
            print("✅ Saved best model!\n")

# ============================================================
# ✅ Run Training
# ============================================================
trained_model = train(model, train_loader, val_loader, device, epochs=3)


✅ Feature size detected: 575


Epoch 1/3: 100%|██████████| 24/24 [03:16<00:00,  8.18s/it, acc=0.0842, loss=2.0442]



Train Loss=4.9550 | Acc=0.0842
Val Acc=0.1250
✅ Saved best model!



Epoch 2/3: 100%|██████████| 24/24 [03:12<00:00,  8.02s/it, acc=0.0947, loss=2.1531]



Train Loss=2.4060 | Acc=0.0947
Val Acc=0.1250


Epoch 3/3: 100%|██████████| 24/24 [03:18<00:00,  8.29s/it, acc=0.1263, loss=2.5811]



Train Loss=2.3955 | Acc=0.1263
Val Acc=0.1250


In [43]:
# ============================================================
# ✅ Data Split & Loaders
# ============================================================
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class MultiModalSLDataset(Dataset):
    def __init__(self, paths, labels, meta, max_frames=100, sample_rate=2):
        self.paths = paths
        self.labels = labels
        self.meta = meta
        self.max_frames = max_frames
        self.sample_rate = sample_rate

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        l = self.labels[idx]
        m = self.meta[idx]
        arr = extract_frame_features(
            p,
            max_frames=self.max_frames,
            sample_rate=self.sample_rate,
            bbox=m.get("bbox", None)
        )
        return torch.tensor(arr, dtype=torch.float32), torch.tensor(l, dtype=torch.long)

# Split data
X_tr, X_val, y_tr, y_val, m_tr, m_val = train_test_split(
    video_paths, labels, meta_info,
    test_size=0.2, stratify=labels, random_state=42
)

train_ds = MultiModalSLDataset(X_tr, y_tr, m_tr)
val_ds   = MultiModalSLDataset(X_val, y_val, m_val)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=0)

# ============================================================
# ✅ LSTM Model
# ============================================================
class SLRecognitionModel(nn.Module):
    def __init__(self, feat_size, hidden=256, num_layers=2, num_classes=None):
        super().__init__()
        self.lstm = nn.LSTM(
            feat_size, hidden,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.5
        )
        self.fc = nn.Linear(hidden * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)

# Infer feature size
dummy = extract_frame_features(video_paths[0], max_frames=10, sample_rate=2, bbox=meta_info[0].get("bbox", None))
feat_size = dummy.shape[1]
print(f"✅ Feature size detected: {feat_size}")

# Model setup
model = SLRecognitionModel(
    feat_size=feat_size,
    hidden=256,
    num_layers=2,
    num_classes=len(gloss_to_idx)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# ============================================================
# ✅ Training Loop (now with 5 epochs)
# ============================================================
def train(model, train_loader, val_loader, device, epochs=5):
    best_acc = 0.0
    for e in range(epochs):
        model.train()
        total_loss, correct, total = 0.0, 0, 0
        loop = tqdm(train_loader, desc=f"Epoch {e+1}/{epochs}", leave=True)
        for X, y in loop:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * X.size(0)
            _, preds = torch.max(logits, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            loop.set_postfix(loss=f"{loss.item():.4f}", acc=f"{correct/total:.4f}")

        epoch_loss = total_loss / total
        epoch_acc = correct / total
        print(f"\nTrain Loss={epoch_loss:.4f} | Acc={epoch_acc:.4f}")

        # validation
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                logits = model(X)
                _, preds = torch.max(logits, 1)
                val_correct += (preds == y).sum().item()
                val_total += y.size(0)
        val_acc = val_correct / val_total
        print(f"Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_multimodal_model.pth")
            print("✅ Saved best model!\n")

# ============================================================
# ✅ Run Training for 5 Epochs
# ============================================================
trained_model = train(model, train_loader, val_loader, device, epochs=5)


✅ Feature size detected: 575


Epoch 1/5: 100%|██████████| 24/24 [03:17<00:00,  8.22s/it, acc=0.0737, loss=1.8353]



Train Loss=4.9424 | Acc=0.0737
Val Acc=0.1250
✅ Saved best model!



Epoch 2/5: 100%|██████████| 24/24 [03:15<00:00,  8.16s/it, acc=0.0526, loss=2.7981]



Train Loss=2.4156 | Acc=0.0526
Val Acc=0.1250


...


Train Loss=2.3625 | Acc=0.0632
Val Acc=0.1250


Epoch 4/5: 100%|██████████| 24/24 [03:16<00:00,  8.20s/it, acc=0.0842, loss=2.2675]



Train Loss=2.3549 | Acc=0.0842
Val Acc=0.1250


Epoch 5/5: 100%|██████████| 24/24 [03:17<00:00,  8.25s/it, acc=0.0947, loss=1.9218]



Train Loss=2.3620 | Acc=0.0947
Val Acc=0.1250


In [2]:
# ============================================================
# ✅ Data Split & Loaders
# ============================================================
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class MultiModalSLDataset(Dataset):
    def __init__(self, paths, labels, meta, max_frames=100, sample_rate=2):
        self.paths = paths
        self.labels = labels
        self.meta = meta
        self.max_frames = max_frames
        self.sample_rate = sample_rate

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        l = self.labels[idx]
        m = self.meta[idx]
        arr = extract_frame_features(
            p,
            max_frames=self.max_frames,
            sample_rate=self.sample_rate,
            bbox=m.get("bbox", None)
        )
        return torch.tensor(arr, dtype=torch.float32), torch.tensor(l, dtype=torch.long)

# Split data
X_tr, X_val, y_tr, y_val, m_tr, m_val = train_test_split(
    video_paths, labels, meta_info,
    test_size=0.2, stratify=labels, random_state=42
)

train_ds = MultiModalSLDataset(X_tr, y_tr, m_tr)
val_ds   = MultiModalSLDataset(X_val, y_val, m_val)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=0)

# ============================================================
# ✅ LSTM Model
# ============================================================
class SLRecognitionModel(nn.Module):
    def __init__(self, feat_size, hidden=256, num_layers=2, num_classes=None):
        super().__init__()
        self.lstm = nn.LSTM(
            feat_size, hidden,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.5
        )
        self.fc = nn.Linear(hidden * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)

# Infer feature size
dummy = extract_frame_features(video_paths[0], max_frames=10, sample_rate=2, bbox=meta_info[0].get("bbox", None))
feat_size = dummy.shape[1]
print(f"✅ Feature size detected: {feat_size}")

# Model setup
model = SLRecognitionModel(
    feat_size=feat_size,
    hidden=256,
    num_layers=2,
    num_classes=len(gloss_to_idx)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# ============================================================
# ✅ Training Loop (now with 5 epochs)
# ============================================================
def train(model, train_loader, val_loader, device, epochs=5):
    best_acc = 0.0
    for e in range(epochs):
        model.train()
        total_loss, correct, total = 0.0, 0, 0
        loop = tqdm(train_loader, desc=f"Epoch {e+1}/{epochs}", leave=True)
        for X, y in loop:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * X.size(0)
            _, preds = torch.max(logits, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            loop.set_postfix(loss=f"{loss.item():.4f}", acc=f"{correct/total:.4f}")

        epoch_loss = total_loss / total
        epoch_acc = correct / total
        print(f"\nTrain Loss={epoch_loss:.4f} | Acc={epoch_acc:.4f}")

        # validation
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                logits = model(X)
                _, preds = torch.max(logits, 1)
                val_correct += (preds == y).sum().item()
                val_total += y.size(0)
        val_acc = val_correct / val_total
        print(f"Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_multimodal_model.pth")
            print("✅ Saved best model!\n")

# ============================================================
# ✅ Run Training for 20 Epochs
# ============================================================
trained_model = train(model, train_loader, val_loader, device, epochs=20)


NameError: name 'video_paths' is not defined

In [3]:
import os, json

# Ensure paths are set correctly
videos_dir = os.path.join(wlasl_path, "videos")
json_file = os.path.join(wlasl_path, "WLASL_v0.3.json")
class_list_file = os.path.join(wlasl_path, "wlasl_class_list.txt")

# Load gloss → index mapping
gloss_to_idx = {}
with open(class_list_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        gloss = line[1:].strip()
        if gloss:
            gloss_to_idx[gloss] = idx
print(f"✅ Loaded {len(gloss_to_idx)} gloss classes.")

# Load JSON annotation
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"✅ Loaded {len(data)} gloss entries.")

# Collect videos and labels
video_paths, labels, meta_info = [], [], []
for entry in data:
    gloss = entry.get("gloss")
    if gloss not in gloss_to_idx:
        continue
    class_idx = gloss_to_idx[gloss]
    for inst in entry.get("instances", []):
        vid = inst.get("video_id")
        fname = vid + ".mp4"
        fullpath = os.path.join(videos_dir, fname)
        if not os.path.isfile(fullpath):
            alt = vid.zfill(5) + ".mp4"
            altpath = os.path.join(videos_dir, alt)
            if os.path.isfile(altpath):
                fullpath = altpath
            else:
                continue
        video_paths.append(fullpath)
        labels.append(class_idx)
        meta_info.append({
            "video_id": vid,
            "gloss": gloss,
            "class_idx": class_idx,
            "frame_start": inst.get("frame_start", 0),
            "frame_end": inst.get("frame_end", None),
            "bbox": inst.get("bbox", None)
        })

print(f"✅ Collected {len(video_paths)} videos and {len(labels)} labels.")


NameError: name 'wlasl_path' is not defined

In [4]:
# ============================================================
# ✅ Setup & Dataset Initialization (from scratch)
# ============================================================
import os, json, kagglehub

# Download / locate the dataset
wlasl_path = kagglehub.dataset_download("risangbaskoro/wlasl-processed")
print("✅ Path to dataset files:", wlasl_path)

# Define file paths
videos_dir = os.path.join(wlasl_path, "videos")
json_file = os.path.join(wlasl_path, "WLASL_v0.3.json")
class_list_file = os.path.join(wlasl_path, "wlasl_class_list.txt")

# Load gloss → index mapping
gloss_to_idx = {}
with open(class_list_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        gloss = line[1:].strip()
        if gloss:
            gloss_to_idx[gloss] = idx
print(f"✅ Loaded {len(gloss_to_idx)} gloss classes.")

# Load JSON annotation
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"✅ Loaded {len(data)} gloss entries.")

# Build lists of video paths, labels, and metadata
video_paths, labels, meta_info = [], [], []
for entry in data:
    gloss = entry.get("gloss")
    if gloss not in gloss_to_idx:
        continue
    class_idx = gloss_to_idx[gloss]
    for inst in entry.get("instances", []):
        vid = inst.get("video_id")
        fname = vid + ".mp4"
        fullpath = os.path.join(videos_dir, fname)
        if not os.path.isfile(fullpath):
            alt = vid.zfill(5) + ".mp4"
            altpath = os.path.join(videos_dir, alt)
            if os.path.isfile(altpath):
                fullpath = altpath
            else:
                continue
        video_paths.append(fullpath)
        labels.append(class_idx)
        meta_info.append({
            "video_id": vid,
            "gloss": gloss,
            "class_idx": class_idx,
            "frame_start": inst.get("frame_start", 0),
            "frame_end": inst.get("frame_end", None),
            "bbox": inst.get("bbox", None)
        })

print(f"✅ Collected {len(video_paths)} videos and {len(labels)} labels.")
print(f"🧩 Example video: {video_paths[0] if len(video_paths)>0 else 'No videos found!'}")


✅ Path to dataset files: /kaggle/input/wlasl-processed
✅ Loaded 2000 gloss classes.
✅ Loaded 2000 gloss entries.
✅ Collected 119 videos and 119 labels.
🧩 Example video: /kaggle/input/wlasl-processed/videos/69241.mp4


In [5]:
# ============================================================
# ✅ Imports
# ============================================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np

# ============================================================
# ✅ Dataset Class
# ============================================================
class MultiModalSLDataset(Dataset):
    def __init__(self, paths, labels, meta, max_frames=100, sample_rate=2):
        self.paths = paths
        self.labels = labels
        self.meta = meta
        self.max_frames = max_frames
        self.sample_rate = sample_rate

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        l = self.labels[idx]
        m = self.meta[idx]
        arr = extract_frame_features(
            p,
            max_frames=self.max_frames,
            sample_rate=self.sample_rate,
            bbox=m.get("bbox", None)
        )
        return torch.tensor(arr, dtype=torch.float32), torch.tensor(l, dtype=torch.long)

# ============================================================
# ✅ Split Dataset
# ============================================================
X_tr, X_val, y_tr, y_val, m_tr, m_val = train_test_split(
    video_paths, labels, meta_info,
    test_size=0.2, stratify=labels, random_state=42
)

train_ds = MultiModalSLDataset(X_tr, y_tr, m_tr)
val_ds   = MultiModalSLDataset(X_val, y_val, m_val)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=0)

print(f"✅ Training videos: {len(train_ds)} | Validation videos: {len(val_ds)}")

# ============================================================
# ✅ Model Definition (LSTM)
# ============================================================
class SLRecognitionModel(nn.Module):
    def __init__(self, feat_size, hidden=256, num_layers=2, num_classes=None):
        super().__init__()
        self.lstm = nn.LSTM(
            feat_size, hidden,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.5
        )
        self.fc = nn.Linear(hidden * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)

# ============================================================
# ✅ Infer Feature Size
# ============================================================
dummy = extract_frame_features(video_paths[0], max_frames=10, sample_rate=2, bbox=meta_info[0].get("bbox", None))
feat_size = dummy.shape[1]
print(f"✅ Feature vector size: {feat_size}")

# ============================================================
# ✅ Initialize Model & Optimizer
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Training on: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")

model = SLRecognitionModel(
    feat_size=feat_size,
    hidden=256,
    num_layers=2,
    num_classes=len(gloss_to_idx)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# ============================================================
# ✅ Training Loop — 20 Epochs
# ============================================================
def train(model, train_loader, val_loader, device, epochs=20):
    best_acc = 0.0
    train_losses, val_accuracies = [], []

    for e in range(epochs):
        model.train()
        total_loss, correct, total = 0.0, 0, 0
        loop = tqdm(train_loader, desc=f"Epoch {e+1}/{epochs}", leave=True)

        for X, y in loop:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * X.size(0)
            _, preds = torch.max(logits, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            loop.set_postfix(loss=f"{loss.item():.4f}", acc=f"{correct/total:.4f}")

        epoch_loss = total_loss / total
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        print(f"\n📉 Train Loss: {epoch_loss:.4f} | 🧠 Train Acc: {epoch_acc:.4f}")

        # Validation
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                logits = model(X)
                _, preds = torch.max(logits, 1)
                val_correct += (preds == y).sum().item()
                val_total += y.size(0)
        val_acc = val_correct / val_total
        val_accuracies.append(val_acc)
        print(f"✅ Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_multimodal_model.pth")
            print("💾 Saved best model!\n")

    return train_losses, val_accuracies

# ============================================================
# 🚀 Run Training for 20 Epochs
# ============================================================
train_losses, val_accuracies = train(model, train_loader, val_loader, device, epochs=20)


✅ Training videos: 95 | Validation videos: 24


NameError: name 'extract_frame_features' is not defined

In [6]:
# ============================================================
# ✅ Feature Extraction Function (MediaPipe + ResNet18)
# ============================================================
import mediapipe as mp
import torchvision.models as models
import torchvision.transforms as T
import cv2
from PIL import Image

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands_det = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device (main): {device}")

# Try initializing CNN safely on GPU, fallback to CPU if needed
cnn_device = device
try:
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    cnn_model.fc = nn.Identity()
    cnn_model = cnn_model.eval().to(cnn_device)
    print(f"✅ ResNet ready on {cnn_device}: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
except Exception as e:
    print("⚠️ CUDA prewarm failed, moving CNN to CPU.")
    cnn_device = torch.device("cpu")
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    cnn_model.fc = nn.Identity()
    cnn_model = cnn_model.eval().to(cnn_device)
    print("✅ CNN ready on CPU.")

# Preprocessing transforms
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# ============================================================
# ✅ Feature Extraction per Video
# ============================================================
def extract_frame_features(video_path, max_frames=100, sample_rate=2, bbox=None):
    cap = cv2.VideoCapture(video_path)
    frames = []
    idx, collected = 0, 0

    while cap.isOpened() and collected < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if idx % sample_rate == 0:
            # Convert to RGB for CNN
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_t = transform(img_pil).unsqueeze(0).to(cnn_device)

            with torch.no_grad():
                feat = cnn_model(img_t).detach().cpu().numpy().reshape(-1)

            # Hand landmarks
            if bbox is not None:
                x1, y1, x2, y2 = map(int, bbox)
                frame_crop = frame[y1:y2, x1:x2]
            else:
                frame_crop = frame

            crop_rgb = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2RGB)
            results = hands_det.process(crop_rgb)

            if results.multi_hand_landmarks:
                lm = results.multi_hand_landmarks[0]
                kp = []
                for l in lm.landmark:
                    kp.extend([l.x, l.y, l.z])
            else:
                kp = [0.0] * 63  # 21 landmarks × 3

            feat_and_kp = np.concatenate([feat, kp], axis=0)
            frames.append(feat_and_kp)
            collected += 1
        idx += 1

    cap.release()

    # Pad/truncate to fixed length
    if len(frames) == 0:
        frames = [np.zeros((512 + 63))] * max_frames
    arr = np.array(frames, dtype=np.float32)
    if arr.shape[0] < max_frames:
        pad_amt = max_frames - arr.shape[0]
        pad_arr = np.zeros((pad_amt, arr.shape[1]), dtype=np.float32)
        arr = np.vstack([arr, pad_arr])
    else:
        arr = arr[:max_frames]

    return arr

print("✅ Feature extractor ready.")


ModuleNotFoundError: No module named 'mediapipe'

In [3]:
!pip install mediapipe==0.10.14 --quiet


In [4]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image
import mediapipe as mp


ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)